<a href="https://colab.research.google.com/github/teteumac/Acoplamento-Quartico-Anomalo/blob/main/CatBoost_MulticlassClassifier_WWCEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

verificar se precisa fazer a normalização

arrumar o smearing

treinar com features menores

In [ ]:
!python3 -m pip install mplhep coffea scikit-optimize
!pip install matplotlib==3.1.3
import lightgbm as lgb
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import mplhep as hep
from matplotlib.colors import ListedColormap
from sklearn.metrics import make_scorer,fbeta_score,precision_score,recall_score,accuracy_score,log_loss,roc_auc_score,classification_report,f1_score,confusion_matrix,roc_curve,precision_recall_curve,average_precision_score
import sys
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from joblib import dump

     |████████████████████████████████| 163kB 18.7MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
     |████████████████████████████████| 5.4MB 23.0MB/s 
     |████████████████████████████████| 10.3MB 30.7MB/s 
     |████████████████████████████████| 8.1MB 21.8MB/s 
     |████████████████████████████████| 245kB 49.9MB/s 
     |████████████████████████████████| 122kB 49.5MB/s 
     |████████████████████████████████| 1.8MB 43.3MB/s 
     |████████████████████████████████| 92kB 9.7MB/s 
     |████████████████████████████████| 1.9MB 36.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
     |████████████████████████████████| 13.1MB 9.7MB/s 
ERROR: mplhep 0.3.7 has requirement matplotlib>=3.4, but you'll have matplotlib 3.1.3 which is incompatible.
ERROR: albumentations 0.1.1

In [ ]:
!pip3 install -q catboost
import catboost
from catboost import CatBoostClassifier

     |████████████████████████████████| 67.3MB 75kB/s 


In [ ]:
%matplotlib inline

In [ ]:
plt.style.use(hep.style.ROOT)

In [ ]:
!pip install matplotlib==3.1.3

In [ ]:
PATH = '/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/'
import time

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def open_file_MC( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2', 'Mx', 'Yx','Mww/Mx', 'Yww_Yx', 'weight'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi 
        return dataframe

def open_file_DD( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]        
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt',
       'jetAK8_eta', 'jetAK8_prunedMass', 'jetAK8_tau21', 'METPt', 'muon_pt',
       'muon_eta', 'ExtraTracks', 'Yww', 'xi1', 'xi2', 'MultiRP1', 'MultiRP2',
       'Mx', 'Yx', 'Mww/Mx', 'Yww_Yx', 'weight'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi       
        return dataframe     

def open_file_Data( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'Yww', 'xi1', 'xi2', 'Mx', 'Yx','Mww/Mx', 'Yww_Yx', 'weight'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi 
        return dataframe        

In [ ]:
select_columns = ['Mww', 'Pt_W_lep', 'Acoplanaridade_Whad_Wlep', 'Acoplanaridade_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass', 'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'Mx',  'Mww/Mx']

In [ ]:
SM =       open_file_MC( PATH + 'DataSet_SM_multiRP.h5')
ANOMALO1 = open_file_MC( PATH + 'DataSet_ANOMALO1_multiRP.h5')
ANOMALO2 = open_file_MC( PATH + 'DataSet_ANOMALO2_multiRP.h5')
ANOMALO3 = open_file_MC( PATH + 'DataSet_ANOMALO3_multiRP.h5')
ANOMALO4 = open_file_MC( PATH + 'DataSet_ANOMALO4_multiRP.h5')
ANOMALO5 = open_file_MC( PATH + 'DataSet_ANOMALO5_multiRP.h5')
ANOMALO6 = open_file_MC( PATH + 'DataSet_ANOMALO6_multiRP.h5')
ANOMALO7 = open_file_MC( PATH + 'DataSet_ANOMALO7_multiRP.h5')
ANOMALO8 = open_file_MC( PATH + 'DataSet_ANOMALO8_multiRP.h5')

label_signal1  = pd.DataFrame( [1]*len( ANOMALO1 ) )
label_signal2  = pd.DataFrame( [1]*len( ANOMALO2 ) )
label_signal3  = pd.DataFrame( [1]*len( ANOMALO3 ) )
label_signal4  = pd.DataFrame( [1]*len( ANOMALO4 ) )
label_signal5  = pd.DataFrame( [1]*len( ANOMALO5 ) )
label_signal6  = pd.DataFrame( [1]*len( ANOMALO6 ) )
label_signal7  = pd.DataFrame( [1]*len( ANOMALO7 ) )
label_signal8  = pd.DataFrame( [1]*len( ANOMALO8 ) )
label_signalSM = pd.DataFrame( [2]*len( SM ) )

In [ ]:
SM = pd.concat( [ SM, label_signalSM ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Standard Model --> ', SM.shape)

ANOMALO1 = pd.concat( [ ANOMALO1, label_signal1 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 1 --> ', ANOMALO1.shape)

ANOMALO2 = pd.concat( [ ANOMALO2, label_signal2 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 2 --> ', ANOMALO2.shape)

ANOMALO3 = pd.concat( [ ANOMALO3, label_signal3 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 3 --> ', ANOMALO3.shape)

ANOMALO4 = pd.concat( [ ANOMALO4, label_signal4 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 4 --> ', ANOMALO4.shape)

ANOMALO5 = pd.concat( [ ANOMALO5, label_signal5 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 5 --> ', ANOMALO5.shape)

ANOMALO6 = pd.concat( [ ANOMALO6, label_signal6 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 6 --> ', ANOMALO6.shape)

ANOMALO7 = pd.concat( [ ANOMALO7, label_signal7 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 7 --> ', ANOMALO7.shape)

ANOMALO8 = pd.concat( [ ANOMALO8, label_signal8 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 8 --> ', ANOMALO8.shape)

Shape for Standard Model -->  (112, 24)
Shape for Anomalo 1 -->  (1544, 24)
Shape for Anomalo 2 -->  (174, 24)
Shape for Anomalo 3 -->  (434, 24)
Shape for Anomalo 4 -->  (742, 24)
Shape for Anomalo 5 -->  (169, 24)
Shape for Anomalo 6 -->  (299, 24)
Shape for Anomalo 7 -->  (672, 24)
Shape for Anomalo 8 -->  (1627, 24)


# Smearing para os xi's dos protons

alterar a composição dos xi's smearing. 

In [ ]:
from random import gauss

def xi_smearing(dset):
    df_xi1 = []
    df_xi2 = []
    for i in range(0,len(dset)):
        df_xi1.append( gauss( 0, 0.1 * dset['xi1'][i] ) + np.array( dset['xi1'][i] ) )
        df_xi2.append( gauss( 0, 0.1 * dset['xi2'][i] ) + np.array( dset['xi2'][i] ) )
    xi1_smearing = pd.DataFrame( df_xi1, columns = ['xi1_smearing'] ) 
    xi2_smearing = pd.DataFrame( df_xi2, columns = ['xi2_smearing'] ) 
    return pd.concat( [ xi1_smearing, xi2_smearing ], axis = 1 )

In [ ]:
'''SM = pd.concat( [ SM, xi_smearing(SM) ], axis = 1 )
ANOMALO1 = pd.concat( [ANOMALO1, xi_smearing(ANOMALO1) ], axis = 1 )
ANOMALO2 = pd.concat( [ANOMALO2, xi_smearing(ANOMALO2) ], axis = 1 )
ANOMALO3 = pd.concat( [ANOMALO3, xi_smearing(ANOMALO3) ], axis = 1 )
ANOMALO4 = pd.concat( [ANOMALO4, xi_smearing(ANOMALO4) ], axis = 1 )
ANOMALO5 = pd.concat( [ANOMALO5, xi_smearing(ANOMALO5) ], axis = 1 )
ANOMALO6 = pd.concat( [ANOMALO6, xi_smearing(ANOMALO6) ], axis = 1 )
ANOMALO7 = pd.concat( [ANOMALO7, xi_smearing(ANOMALO7) ], axis = 1 )
ANOMALO8 = pd.concat( [ANOMALO8, xi_smearing(ANOMALO8) ], axis = 1 )'''

'SM = pd.concat( [ SM, xi_smearing(SM) ], axis = 1 )\nANOMALO1 = pd.concat( [ANOMALO1, xi_smearing(ANOMALO1) ], axis = 1 )\nANOMALO2 = pd.concat( [ANOMALO2, xi_smearing(ANOMALO2) ], axis = 1 )\nANOMALO3 = pd.concat( [ANOMALO3, xi_smearing(ANOMALO3) ], axis = 1 )\nANOMALO4 = pd.concat( [ANOMALO4, xi_smearing(ANOMALO4) ], axis = 1 )\nANOMALO5 = pd.concat( [ANOMALO5, xi_smearing(ANOMALO5) ], axis = 1 )\nANOMALO6 = pd.concat( [ANOMALO6, xi_smearing(ANOMALO6) ], axis = 1 )\nANOMALO7 = pd.concat( [ANOMALO7, xi_smearing(ANOMALO7) ], axis = 1 )\nANOMALO8 = pd.concat( [ANOMALO8, xi_smearing(ANOMALO8) ], axis = 1 )'

In [ ]:
'''DataSet_multiRP_DrellYan = open_file_MC(  PATH +'DataSet_multiRP_DrellYan.h5' )
DataSet_multiRP_QCD = open_file_MC(  PATH +'DataSet_multiRP_QCD.h5' )
DataSet_multiRP_single_top = open_file_MC( PATH + 'DataSet_multiRP_single_top.h5' )
DataSet_multiRP_TTbar = open_file_MC(  PATH +'DataSet_multiRP_TTbar.h5' )
DataSet_multiRP_VV_inclusivo = open_file_MC( PATH + 'DataSet_multiRP_VV_inclusivo.h5' )
DataSet_multiRP_WJets = open_file_MC( PATH + 'DataSet_multiRP_WJets.h5' )'''

"DataSet_multiRP_DrellYan = open_file_MC(  PATH +'DataSet_multiRP_DrellYan.h5' )\nDataSet_multiRP_QCD = open_file_MC(  PATH +'DataSet_multiRP_QCD.h5' )\nDataSet_multiRP_single_top = open_file_MC( PATH + 'DataSet_multiRP_single_top.h5' )\nDataSet_multiRP_TTbar = open_file_MC(  PATH +'DataSet_multiRP_TTbar.h5' )\nDataSet_multiRP_VV_inclusivo = open_file_MC( PATH + 'DataSet_multiRP_VV_inclusivo.h5' )\nDataSet_multiRP_WJets = open_file_MC( PATH + 'DataSet_multiRP_WJets.h5' )"

In [ ]:
'''label_DrellYan = pd.DataFrame( [0]*len( DataSet_multiRP_DrellYan ) )
DataSet_multiRP_DrellYan = pd.concat( [ DataSet_multiRP_DrellYan, label_DrellYan ], axis = 1 ).rename(columns={0: 'label'})

label_QCD = pd.DataFrame( [0]*len( DataSet_multiRP_QCD ) )
DataSet_multiRP_QCD = pd.concat( [ DataSet_multiRP_QCD, label_QCD ], axis = 1 ).rename(columns={0: 'label'})

label_SingleTop = pd.DataFrame( [0]*len( DataSet_multiRP_single_top ) )
DataSet_multiRP_single_top = pd.concat( [ DataSet_multiRP_single_top, label_SingleTop ], axis = 1 ).rename(columns={0: 'label'})

label_TTbar = pd.DataFrame( [0]*len( DataSet_multiRP_TTbar ) )
DataSet_multiRP_TTbar = pd.concat( [ DataSet_multiRP_TTbar, label_TTbar ], axis = 1 ).rename(columns={0: 'label'})

label_VVInclusive = pd.DataFrame( [0]*len( DataSet_multiRP_VV_inclusivo ) )
DataSet_multiRP_VV_inclusivo = pd.concat( [ DataSet_multiRP_VV_inclusivo, label_VVInclusive ], axis = 1 ).rename(columns={0: 'label'})

label_WJets = pd.DataFrame( [0]*len( DataSet_multiRP_WJets ) )
DataSet_multiRP_WJets = pd.concat( [ DataSet_multiRP_WJets, label_WJets ], axis = 1 ).rename(columns={0: 'label'})'''

"label_DrellYan = pd.DataFrame( [0]*len( DataSet_multiRP_DrellYan ) )\nDataSet_multiRP_DrellYan = pd.concat( [ DataSet_multiRP_DrellYan, label_DrellYan ], axis = 1 ).rename(columns={0: 'label'})\n\nlabel_QCD = pd.DataFrame( [0]*len( DataSet_multiRP_QCD ) )\nDataSet_multiRP_QCD = pd.concat( [ DataSet_multiRP_QCD, label_QCD ], axis = 1 ).rename(columns={0: 'label'})\n\nlabel_SingleTop = pd.DataFrame( [0]*len( DataSet_multiRP_single_top ) )\nDataSet_multiRP_single_top = pd.concat( [ DataSet_multiRP_single_top, label_SingleTop ], axis = 1 ).rename(columns={0: 'label'})\n\nlabel_TTbar = pd.DataFrame( [0]*len( DataSet_multiRP_TTbar ) )\nDataSet_multiRP_TTbar = pd.concat( [ DataSet_multiRP_TTbar, label_TTbar ], axis = 1 ).rename(columns={0: 'label'})\n\nlabel_VVInclusive = pd.DataFrame( [0]*len( DataSet_multiRP_VV_inclusivo ) )\nDataSet_multiRP_VV_inclusivo = pd.concat( [ DataSet_multiRP_VV_inclusivo, label_VVInclusive ], axis = 1 ).rename(columns={0: 'label'})\n\nlabel_WJets = pd.DataFrame( 

In [ ]:
data_set_back_multirp = open_file_DD( PATH + 'DataDriven_Background_multiRP.h5' )

In [ ]:
label_back = pd.DataFrame( [0]*len( data_set_back_multirp ) )
data_set_back_multirp = pd.concat( [ data_set_back_multirp, label_back ], axis = 1 ).rename(columns={0: 'label'})
data_set_back_multirp

,Mww,Pt_W_lep,dPhi_Whad_Wlep,dPhi_jatos_MET,jetAK8_pt,jetAK8_eta,jetAK8_prunedMass,jetAK8_tau21,METPt,muon_pt,muon_eta,ExtraTracks,Yww,xi1,xi2,MultiRP1,MultiRP2,Mx,Yx,Mww/Mx,Yww_Yx,weight,Acoplanaridade_Whad_Wlep,Acoplanaridade_jatos_MET,label
0,706.608342,229.433921,3.104932,3.016140,244.365721,0.758463,8.573437,0.735708,42.810390,186.830750,1.003712,34.0,1.106191,0.052109,0.111753,1.0,1.0,992.039526,0.381479,0.712278,0.724713,0.01,0.011669,0.039933,0
1,736.491948,329.974506,3.067420,2.368258,369.061507,1.535922,16.263502,0.706058,97.642898,262.862366,1.575521,42.0,1.494392,0.077207,0.113172,1.0,1.0,1215.179468,0.191209,0.606077,1.303183,0.01,0.023610,0.246160,0
2,1012.055130,250.191224,-2.989816,3.101588,447.352242,0.476539,7.604977,0.855670,197.791872,67.529274,1.426694,46.0,1.392727,0.099899,0.120155,1.0,1.0,1424.281045,0.092312,0.710573,1.300415,0.01,0.048312,0.012734,0
3,801.627682,401.144449,2.196309,2.259357,394.456184,1.852053,8.174912,0.834630,135.615686,265.935516,0.936441,48.0,0.957040,0.043641,0.091501,1.0,1.0,821.491532,0.370178,0.975820,0.586861,0.01,0.300893,0.280824,0
4,956.729620,344.981106,2.988486,2.411655,526.427156,0.508421,171.405609,0.320342,91.846592,272.636658,0.215901,38.0,0.207985,0.102238,0.042712,1.0,1.0,859.065180,-0.436410,1.113687,0.644395,0.01,0.048735,0.232346,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417340,670.308551,235.492546,-2.849825,-2.691901,300.836813,1.093756,18.986254,0.844727,186.298483,59.261715,0.238643,27.0,0.251635,0.100328,0.043516,1.0,1.0,858.969850,-0.417659,0.780363,0.669293,0.01,0.092873,0.143141,0
417341,1160.930547,243.647700,2.740906,2.720288,217.646715,0.780753,65.993027,0.417648,139.734327,103.982986,1.947919,50.0,2.293765,0.073859,0.100618,1.0,1.0,1120.683552,0.154591,1.035913,2.139174,0.01,0.127542,0.134105,0
417342,715.553066,228.009132,-3.040333,-2.828814,201.868402,0.601398,7.845040,0.798828,187.275407,59.687149,1.564484,64.0,-1.481526,0.072363,0.066277,1.0,1.0,900.291512,-0.043930,0.794802,-1.437596,0.01,0.032232,0.099561,0
417343,708.358903,108.294984,-2.476245,-2.448281,234.299604,0.933830,66.672958,0.647939,54.611703,53.726337,2.363542,53.0,-1.742363,0.056347,0.118468,1.0,1.0,1062.134849,0.371552,0.666920,-2.113915,0.01,0.211787,0.220688,0


In [ ]:
Dataset_Signal_Back1  = pd.concat( [ ANOMALO1 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back2  = pd.concat( [ ANOMALO2 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back3  = pd.concat( [ ANOMALO3 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back4  = pd.concat( [ ANOMALO4 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back5  = pd.concat( [ ANOMALO5 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back6  = pd.concat( [ ANOMALO6 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back7  = pd.concat( [ ANOMALO7 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back8  = pd.concat( [ ANOMALO8 , data_set_back_multirp, SM  ], axis = 0, sort = False )

In [ ]:
data_set_dados_multirp = open_file_Data( PATH + 'DataSet_dados_multiRP.h5' )
data_set_dados_multirp

,Mww,Pt_W_lep,dPhi_Whad_Wlep,dPhi_jatos_MET,jetAK8_pt,jetAK8_eta,jetAK8_prunedMass,jetAK8_tau21,METPt,muon_pt,muon_eta,ExtraTracks,Yww,xi1,xi2,Mx,Yx,Mww/Mx,Yww_Yx,weight,Acoplanaridade_Whad_Wlep,Acoplanaridade_jatos_MET
0,741.827618,213.047177,3.010690,1.859894,226.343484,0.390933,74.849960,0.565876,57.706423,196.705261,1.649120,34.0,1.269749e-321,-1.482681,0.105611,0.073181,1142.870196,-0.183416,0.649092,-1.299265,0.041668,0.407977
1,877.505354,123.897340,2.012689,2.549330,233.792329,0.997459,32.459290,0.487617,92.757254,64.812515,2.137774,33.0,1.269749e-321,1.974074,0.059030,0.074471,861.936296,0.116178,1.018063,1.857896,0.359341,0.188523
2,781.795723,350.697573,-3.013293,-2.860292,402.634166,1.507299,37.748211,0.642403,259.948342,101.810310,1.184208,39.0,0.000000e+00,1.168085,0.099538,0.095934,1270.353032,-0.018436,0.615416,1.186522,0.040839,0.089541
3,650.542403,294.405514,-3.018462,2.739984,253.529887,0.734596,87.312218,0.249258,96.085753,216.662903,0.263637,70.0,0.000000e+00,-0.254712,0.085283,0.073905,1032.072127,-0.071600,0.630326,-0.183113,0.039194,0.127836
4,1251.788316,604.512709,-3.082175,3.061210,566.258455,0.385871,164.440262,0.215055,303.104532,307.281799,0.911545,45.0,0.000000e+00,-0.910245,0.085583,0.092613,1157.367409,0.039472,1.081582,-0.949717,0.018913,0.025586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4212,748.899085,201.394726,3.077837,-3.125663,488.959260,1.246388,7.140489,0.767577,148.520666,54.639683,0.050968,91.0,0.000000e+00,0.064142,0.091740,0.115538,1338.392770,0.115320,0.559551,-0.051178,0.020294,0.005071
4213,1595.418427,276.905510,-2.768412,-2.916644,425.321662,2.213353,137.410568,0.447905,170.338762,111.314743,0.700486,77.0,0.000000e+00,0.736542,0.041849,0.112344,891.372274,0.493753,1.789845,0.242789,0.118787,0.071604
4214,958.358525,239.519734,3.077765,2.677275,229.409691,0.436397,9.861922,0.855264,50.609485,193.921387,2.053018,28.0,0.000000e+00,2.178533,0.074463,0.129167,1274.936342,0.275401,0.751691,1.903132,0.020317,0.147797
4215,905.681403,410.041595,-2.741795,-2.756343,353.064677,1.242856,47.823807,0.656382,276.760091,133.371582,0.005376,25.0,0.000000e+00,-0.005690,0.090407,0.107519,1281.699892,0.086676,0.706625,-0.092366,0.127260,0.122629


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
test_size = 0.35
DataSet_Train1_, DataSet_Test1_ = train_test_split( Dataset_Signal_Back1, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back1.label )
DataSet_Train2_, DataSet_Test2_ = train_test_split( Dataset_Signal_Back2, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back2.label )
DataSet_Train3_, DataSet_Test3_ = train_test_split( Dataset_Signal_Back3, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back3.label )
DataSet_Train4_, DataSet_Test4_ = train_test_split( Dataset_Signal_Back4, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back4.label )
DataSet_Train5_, DataSet_Test5_ = train_test_split( Dataset_Signal_Back5, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back5.label )
DataSet_Train6_, DataSet_Test6_ = train_test_split( Dataset_Signal_Back6, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back6.label )
DataSet_Train7_, DataSet_Test7_ = train_test_split( Dataset_Signal_Back7, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back7.label )
DataSet_Train8_, DataSet_Test8_ = train_test_split( Dataset_Signal_Back8, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back8.label )

In [ ]:
'''with h5py.File( 'DataSet_Test1_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test1_ )
with h5py.File( 'DataSet_Test2_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test2_ )
with h5py.File( 'DataSet_Test3_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test3_ ) 
with h5py.File( 'DataSet_Test4_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test4_ )
with h5py.File( 'DataSet_Test5_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test5_ )    
with h5py.File( 'DataSet_Test6_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test6_ )
with h5py.File( 'DataSet_Test7_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test7_ )
with h5py.File( 'DataSet_Test8_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test8_ )    


with h5py.File( 'DataSet_Train1_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Train1_ )
with h5py.File( 'DataSet_Train2_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Train2_ )
with h5py.File( 'DataSet_Train3_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Train3_ ) 
with h5py.File( 'DataSet_Train4_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Train4_ )
with h5py.File( 'DataSet_Train5_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Train5_ )    
with h5py.File( 'DataSet_Train6_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Train6_ )
with h5py.File( 'DataSet_Train7_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Train7_ )
with h5py.File( 'DataSet_Train8_.h5', 'w') as f:
    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Train8_ )    '''

"with h5py.File( 'DataSet_Test1_.h5', 'w') as f:\n    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test1_ )\nwith h5py.File( 'DataSet_Test2_.h5', 'w') as f:\n    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test2_ )\nwith h5py.File( 'DataSet_Test3_.h5', 'w') as f:\n    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test3_ ) \nwith h5py.File( 'DataSet_Test4_.h5', 'w') as f:\n    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test4_ )\nwith h5py.File( 'DataSet_Test5_.h5', 'w') as f:\n    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test5_ )    \nwith h5py.File( 'DataSet_Test6_.h5', 'w') as f:\n    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test6_ )\nwith h5py.File( 'DataSet_Test7_.h5', 'w') as f:\n    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test7_ )\nwith h5py.File( 'DataSet_Test8_.h5', 'w') as f:\n    DataSet_Test5_ = f.create_dataset( 'dados', data = DataSet_Test8_ )    \n\n\nwith h5py.

In [ ]:
y_train1 = DataSet_Train1_['label']
y_test1  = DataSet_Test1_['label']

y_train2 = DataSet_Train2_['label']
y_test2  = DataSet_Test2_['label']

y_train3 = DataSet_Train3_['label']
y_test3  = DataSet_Test3_['label']

y_train4 = DataSet_Train4_['label']
y_test4  = DataSet_Test4_['label']

y_train5 = DataSet_Train5_['label']
y_test5  = DataSet_Test5_['label']

y_train6 = DataSet_Train6_['label']
y_test6  = DataSet_Test6_['label']

y_train7 = DataSet_Train7_['label']
y_test7  = DataSet_Test7_['label']

y_train8 = DataSet_Train8_['label']
y_test8  = DataSet_Test8_['label']

In [ ]:
print('--- Weight Anomalo 8 --- \n')
DataSet_Test8_weight_signal =  DataSet_Test8_[DataSet_Test8_['label']==1]['weight'] 
DataSet_TestSM8_weight_signal =  DataSet_Test8_[DataSet_Test8_['label']==2]['weight'] 
DataSet_Test8_weight_backgr =DataSet_Test8_[DataSet_Test8_['label']==0]['weight'] 

DataSet_Train8_weight_signal = DataSet_Train8_[DataSet_Train8_['label']==1]['weight']
DataSet_TrainSM8_weight_signal =  DataSet_Train8_[DataSet_Train8_['label']==2]['weight']
DataSet_Train8_weight_backgr = DataSet_Train8_[DataSet_Train8_['label']==0]['weight'] 

DataSet_Test8_weight =  DataSet_Test8_['weight']

print( 'Shape Test Signal', DataSet_Test8_weight_signal.shape)
print( 'Shape Test SM', DataSet_TestSM8_weight_signal.shape)
print( 'Shape Test Background', DataSet_Test8_weight_backgr.shape, '\n')

print('--- Weight Anomalo 7 --- \n')

DataSet_Test7_weight_signal =  DataSet_Test7_[DataSet_Test7_['label']==1]['weight'] 
DataSet_TestSM7_weight_signal =DataSet_Test7_[DataSet_Test7_['label']==2]['weight'] 
DataSet_Test7_weight_backgr =  DataSet_Test7_[DataSet_Test7_['label']==0]['weight'] 

DataSet_Train7_weight_signal =  DataSet_Train7_[DataSet_Train7_['label']==1]['weight']
DataSet_TrainSM7_weight_signal =  DataSet_Train7_[DataSet_Train7_['label']==2]['weight']
DataSet_Train7_weight_backgr =  DataSet_Train7_[DataSet_Train7_['label']==0]['weight'] 

DataSet_Test7_weight =  DataSet_Test7_['weight']

print('--- Weight Anomalo 6 --- \n')

DataSet_Test6_weight_signal =  DataSet_Test6_[DataSet_Test6_['label']==1]['weight'] 
DataSet_TestSM6_weight_signal =DataSet_Test6_[DataSet_Test6_['label']==2]['weight'] 
DataSet_Test6_weight_backgr =  DataSet_Test6_[DataSet_Test6_['label']==0]['weight'] 

DataSet_Train6_weight_signal =  DataSet_Train6_[DataSet_Train6_['label']==1]['weight']
DataSet_TrainSM6_weight_signal =  DataSet_Train6_[DataSet_Train6_['label']==2]['weight']
DataSet_Train6_weight_backgr =  DataSet_Train6_[DataSet_Train6_['label']==0]['weight'] 

DataSet_Test6_weight =  DataSet_Test6_['weight']

print( 'Shape Test Signal',  DataSet_Test7_weight_signal.shape)
print( 'Shape Test SM', DataSet_TestSM7_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test7_weight_backgr.shape, '\n')

print('--- Weight Anomalo 5 --- \n')

DataSet_Test5_weight_signal =  DataSet_Test5_[DataSet_Test5_['label']==1]['weight'] 
DataSet_TestSM5_weight_signal =DataSet_Test5_[DataSet_Test5_['label']==2]['weight'] 
DataSet_Test5_weight_backgr =  DataSet_Test5_[DataSet_Test5_['label']==0]['weight'] 

DataSet_Train5_weight_signal =  DataSet_Train5_[DataSet_Train5_['label']==1]['weight']
DataSet_TrainSM5_weight_signal =  DataSet_Train5_[DataSet_Train5_['label']==2]['weight']
DataSet_Train5_weight_backgr =  DataSet_Train5_[DataSet_Train5_['label']==0]['weight'] 

DataSet_Test5_weight =  DataSet_Test5_['weight']

print( 'Shape Test Signal',  DataSet_Test7_weight_signal.shape)
print( 'Shape Test SM', DataSet_TestSM7_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test7_weight_backgr.shape, '\n')

print('--- Weight Anomalo 1 --- \n')

DataSet_Test1_weight_signal =  DataSet_Test1_[DataSet_Test1_['label']==1]['weight'] 
DataSet_TestSM1_weight_signal =  DataSet_Test1_[DataSet_Test1_['label']==2]['weight'] 
DataSet_Test1_weight_backgr =  DataSet_Test1_[DataSet_Test1_['label']==0]['weight'] 

DataSet_Train1_weight_signal =  DataSet_Train1_[DataSet_Train1_['label']==1]['weight']
DataSet_TrainSM1_weight_signal =  DataSet_Train1_[DataSet_Train1_['label']==2]['weight']
DataSet_Train1_weight_backgr = DataSet_Train1_[DataSet_Train1_['label']==0]['weight'] 

DataSet_Test1_weight =  DataSet_Test1_['weight']

print( 'Shape Test Signal', DataSet_Test1_weight_signal.shape)
print('Shape Test SM',  DataSet_TestSM1_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test1_weight_backgr.shape, '\n')

print('--- Weight Anomalo 2 --- \n')

DataSet_Test2_weight_signal =  DataSet_Test2_[DataSet_Test2_['label']==1]['weight'] 
DataSet_TestSM2_weight_signal =  DataSet_Test2_[DataSet_Test2_['label']==2]['weight'] 
DataSet_Test2_weight_backgr =  DataSet_Test2_[DataSet_Test2_['label']==0]['weight'] 

DataSet_Train2_weight_signal =  DataSet_Train2_[DataSet_Train2_['label']==1]['weight']
DataSet_TrainSM2_weight_signal =  DataSet_Train2_[DataSet_Train2_['label']==2]['weight']
DataSet_Train2_weight_backgr = DataSet_Train2_[DataSet_Train2_['label']==0]['weight'] 

DataSet_Test2_weight =  DataSet_Test2_['weight']

print( 'Shape Test Signal', DataSet_Test2_weight_signal.shape)
print('Shape Test SM',  DataSet_TestSM2_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test2_weight_backgr.shape, '\n')

print('--- Weight Anomalo 3 --- \n')

DataSet_Test3_weight_signal =  DataSet_Test3_[DataSet_Test3_['label']==1]['weight'] 
DataSet_TestSM3_weight_signal =  DataSet_Test3_[DataSet_Test3_['label']==2]['weight'] 
DataSet_Test3_weight_backgr =  DataSet_Test3_[DataSet_Test3_['label']==0]['weight'] 

DataSet_Train3_weight_signal =  DataSet_Train3_[DataSet_Train3_['label']==1]['weight']
DataSet_TrainSM3_weight_signal =  DataSet_Train3_[DataSet_Train3_['label']==2]['weight']
DataSet_Train3_weight_backgr = DataSet_Train3_[DataSet_Train3_['label']==0]['weight'] 

DataSet_Test3_weight =  DataSet_Test2_['weight']

print( 'Shape Test Signal', DataSet_Test3_weight_signal.shape)
print('Shape Test SM',  DataSet_TestSM3_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test3_weight_backgr.shape, '\n')

print('--- Weight Anomalo 4 --- \n')

DataSet_Test4_weight_signal = DataSet_Test4_[DataSet_Test4_['label']==1]['weight'] 
DataSet_TestSM4_weight_signal =  DataSet_Test4_[DataSet_Test4_['label']==2]['weight'] 
DataSet_Test4_weight_backgr = DataSet_Test4_[DataSet_Test4_['label']==0]['weight'] 

DataSet_Train4_weight_signal = DataSet_Train4_[DataSet_Train4_['label']==1]['weight']
DataSet_TrainSM4_weight_signal =  DataSet_Train4_[DataSet_Train4_['label']==2]['weight']
DataSet_Train4_weight_backgr = DataSet_Train4_[DataSet_Train4_['label']==0]['weight'] 

DataSet_Test4_weight =  DataSet_Test4_['weight']

print( 'Shape Test SM' , DataSet_Test4_weight_signal.shape)
print( 'Shape Test SM', DataSet_TestSM4_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test4_weight_backgr.shape)

--- Weight Anomalo 8 --- 

Shape Test Signal (570,)
Shape Test SM (39,)
Shape Test Background (146071,) 

--- Weight Anomalo 7 --- 

--- Weight Anomalo 6 --- 

Shape Test Signal (235,)
Shape Test SM (39,)
Shape Test Background (146072,) 

--- Weight Anomalo 5 --- 

Shape Test Signal (235,)
Shape Test SM (39,)
Shape Test Background (146072,) 

--- Weight Anomalo 1 --- 

Shape Test Signal (541,)
Shape Test SM (39,)
Shape Test Background (146071,) 

--- Weight Anomalo 2 --- 

Shape Test Signal (61,)
Shape Test SM (39,)
Shape Test Background (146071,) 

--- Weight Anomalo 3 --- 

Shape Test Signal (152,)
Shape Test SM (39,)
Shape Test Background (146071,) 

--- Weight Anomalo 4 --- 

Shape Test SM (260,)
Shape Test SM (39,)
Shape Test Background (146071,)


In [ ]:

DataSet_Train8 = DataSet_Train8_[select_columns] 
DataSet_Test8 = DataSet_Test8_[select_columns] 

DataSet_Train7 = DataSet_Train7_[select_columns] 
DataSet_Test7 = DataSet_Test7_[select_columns] 

DataSet_Train6 = DataSet_Train6_[select_columns] 
DataSet_Test6 = DataSet_Test6_[select_columns] 

DataSet_Train5 = DataSet_Train5_[select_columns] 
DataSet_Test5 = DataSet_Test5_[select_columns] 

DataSet_Train1 = DataSet_Train1_[select_columns] 
DataSet_Test1 = DataSet_Test1_[select_columns] 

DataSet_Train2 = DataSet_Train2_[select_columns] 
DataSet_Test2 = DataSet_Test2_[select_columns] 

DataSet_Train3 = DataSet_Train3_[select_columns] 
DataSet_Test3 = DataSet_Test3_[select_columns] 

DataSet_Train4 = DataSet_Train4_[select_columns] 
DataSet_Test4 = DataSet_Test4_[select_columns] 


In [ ]:
scaler1 = StandardScaler()
X_train_scaled1 = scaler1.fit_transform( DataSet_Train1 )
X_test_scaled1 = scaler1.transform( DataSet_Test1 )


scaler2 = StandardScaler()
X_train_scaled2 = scaler2.fit_transform( DataSet_Train2 )
X_test_scaled2 = scaler2.transform( DataSet_Test2 )

scaler3 = StandardScaler()
X_train_scaled3 = scaler3.fit_transform( DataSet_Train3 )
X_test_scaled3 = scaler3.transform( DataSet_Test3 )

scaler4 = StandardScaler()
X_train_scaled4 = scaler4.fit_transform( DataSet_Train4 )
X_test_scaled4 = scaler4.transform( DataSet_Test4 )

scaler5 = StandardScaler()
X_train_scaled5 = scaler5.fit_transform( DataSet_Train5 )
X_test_scaled5 = scaler5.transform( DataSet_Test5 )

scaler6 = StandardScaler()
X_train_scaled6 = scaler6.fit_transform( DataSet_Train6 )
X_test_scaled6 = scaler6.transform( DataSet_Test6 )

scaler7 = StandardScaler()
X_train_scaled7 = scaler7.fit_transform( DataSet_Train7 )
X_test_scaled7 = scaler7.transform( DataSet_Test7 )

scaler8 = StandardScaler()
X_train_scaled8 = scaler8.fit_transform( DataSet_Train8 )
X_test_scaled8 = scaler8.transform( DataSet_Test8 )

In [ ]:

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from lightgbm import LGBMClassifier


In [ ]:
n_iter = 150
cv = 2
scoring = 'f1_weighted'
param_grid = {
    'learning_rate': [0.008, 0.0004,  1],
    'depth': [0,5,15],
    'iterations':[ 600, 1000],
    'l2_leaf_reg':[ 6, 50],
    'border_count':[20, 150]
              }

In [ ]:

print('Escolhendo...')
#mdl8_binary_logloss_scale_pos_weight =  LGBMClassifier( class_weight = { 0: DataSet_Train8_weight_backgr.sum(), 1: DataSet_Train8_weight_signal.sum(), 2:DataSet_TrainSM8_weight_signal.sum() } )
mdl8_binary_logloss_scale_pos_weight =  CatBoostClassifier( loss_function='MultiClass'  )
search8_binary_logloss_scale_pos_weight = GridSearchCV( estimator = mdl8_binary_logloss_scale_pos_weight, param_grid=param_grid ,cv=cv, verbose=3, n_jobs=-1  )
search_result8_binary_logloss_scale_pos_weight = search8_binary_logloss_scale_pos_weight.fit( X_train_scaled8, y_train8 )
print("Best: %f using %s" % ( search_result8_binary_logloss_scale_pos_weight.best_score_, search_result8_binary_logloss_scale_pos_weight.best_params_ ) )
print('\n')
dump(search_result8_binary_logloss_scale_pos_weight,PATH + 'CatBoost_Multiclass_ANOMALO8_DataDriven.joblib') 

print('Escolhendo...')
#mdl6_binary_logloss_scale_pos_weight =  LGBMClassifier( class_weight = { 0: DataSet_Train6_weight_backgr.sum(), 1: DataSet_Train6_weight_signal.sum(), 2:DataSet_TrainSM6_weight_signal.sum() } )
mdl6_binary_logloss_scale_pos_weight = CatBoostClassifier( loss_function='MultiClass')
time_s_ = time.time()
search6_binary_logloss_scale_pos_weight = GridSearchCV( estimator = mdl6_binary_logloss_scale_pos_weight, param_grid=param_grid ,cv=cv, verbose=3, n_jobs=-1   )
search_result6_binary_logloss_scale_pos_weight = search6_binary_logloss_scale_pos_weight.fit( X_train_scaled6, y_train6 )
print("Best: %f using %s" % ( search_result6_binary_logloss_scale_pos_weight.best_score_, search_result6_binary_logloss_scale_pos_weight.best_params_ ) )
print('\n')
dump(search_result6_binary_logloss_scale_pos_weight,PATH + 'CatBoost_Multiclass_ANOMALO6_DataDriven.joblib') 
time_e_ = time.time()
print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

print('Escolhendo...')
#mdl7_binary_logloss_scale_pos_weight =  LGBMClassifier( class_weight = { 0: DataSet_Train7_weight_backgr.sum(), 1: DataSet_Train7_weight_signal.sum(), 2:DataSet_TrainSM7_weight_signal.sum() } )
mdl7_binary_logloss_scale_pos_weight = CatBoostClassifier( loss_function='MultiClass' )
time_s_ = time.time()
search7_binary_logloss_scale_pos_weight = GridSearchCV( estimator = mdl7_binary_logloss_scale_pos_weight, param_grid=param_grid,cv=cv, verbose=3, n_jobs=-1  )
search_result7_binary_logloss_scale_pos_weight = search7_binary_logloss_scale_pos_weight.fit( X_train_scaled7, y_train7 )
print("Best: %f using %s" % ( search_result7_binary_logloss_scale_pos_weight.best_score_, search_result7_binary_logloss_scale_pos_weight.best_params_ ) )
print('\n')
dump(search_result7_binary_logloss_scale_pos_weight,PATH + 'CatBoost_Multiclass_ANOMALO7_DataDriven.joblib') 
time_e_ = time.time()
print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

print('Escolhendo...')
#mdl5_binary_logloss_scale_pos_weight =  LGBMClassifier( class_weight = { 0: DataSet_Train5_weight_backgr.sum(), 1: DataSet_Train5_weight_signal.sum(), 2:DataSet_TrainSM5_weight_signal.sum() } )
mdl5_binary_logloss_scale_pos_weight = CatBoostClassifier( loss_function='MultiClass')
time_s_ = time.time()
search5_binary_logloss_scale_pos_weight = GridSearchCV( estimator = mdl5_binary_logloss_scale_pos_weight, param_grid=param_grid ,cv=cv, verbose=3, n_jobs=-1  )
search_result5_binary_logloss_scale_pos_weight = search5_binary_logloss_scale_pos_weight.fit( X_train_scaled5, y_train5 )
print("Best: %f using %s" % ( search_result5_binary_logloss_scale_pos_weight.best_score_, search_result5_binary_logloss_scale_pos_weight.best_params_ ) )
print('\n')
dump(search_result5_binary_logloss_scale_pos_weight,PATH + 'CatBoost_Multiclass_ANOMALO5_DataDriven.joblib') 
time_e_ = time.time()
print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

print('Escolhendo...')
#mdl1_binary_logloss_scale_pos_weight =  LGBMClassifier( class_weight = { 0: DataSet_Train1_weight_backgr.sum(), 1: DataSet_Train1_weight_signal.sum(), 2:DataSet_TrainSM1_weight_signal.sum() } )
mdl1_binary_logloss_scale_pos_weight = CatBoostClassifier( loss_function='MultiClass' )
time_s_ = time.time()
search1_binary_logloss_scale_pos_weight = GridSearchCV( estimator = mdl1_binary_logloss_scale_pos_weight, param_grid=param_grid ,cv=cv, verbose=3, n_jobs=-1   )
search_result1_binary_logloss_scale_pos_weight = search1_binary_logloss_scale_pos_weight.fit( X_train_scaled1, y_train1 )
print("Best: %f using %s" % ( search_result1_binary_logloss_scale_pos_weight.best_score_, search_result1_binary_logloss_scale_pos_weight.best_params_ ) )
print('\n')
dump(search_result1_binary_logloss_scale_pos_weight,PATH + 'CatBoost_Multiclass_ANOMALO1_DataDriven.joblib') 
time_e_ = time.time()
print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

print('Escolhendo...')
#mdl2_binary_logloss_scale_pos_weight =  LGBMClassifier( class_weight = { 0: DataSet_Train2_weight_backgr.sum(), 1: DataSet_Train2_weight_signal.sum(), 2:DataSet_TrainSM2_weight_signal.sum() } )
mdl2_binary_logloss_scale_pos_weight = CatBoostClassifier( loss_function='MultiClass' )
search2_binary_logloss_scale_pos_weight = GridSearchCV( estimator = mdl2_binary_logloss_scale_pos_weight, param_grid=param_grid,cv=cv, verbose=3, n_jobs=-1  )
search_result2_binary_logloss_scale_pos_weight = search2_binary_logloss_scale_pos_weight.fit( X_train_scaled2, y_train2 )
print("Best: %f using %s" % ( search_result2_binary_logloss_scale_pos_weight.best_score_, search_result2_binary_logloss_scale_pos_weight.best_params_ ) )
print('\n')
dump(search_result2_binary_logloss_scale_pos_weight,PATH + 'CatBoost_Multiclass_ANOMALO2_DataDriven.joblib') 

print('Escolhendo...')
#mdl3_binary_logloss_scale_pos_weight =  LGBMClassifier( class_weight = { 0: DataSet_Train3_weight_backgr.sum(), 1: DataSet_Train3_weight_signal.sum(), 2:DataSet_TrainSM3_weight_signal.sum() } )
mdl3_binary_logloss_scale_pos_weight = CatBoostClassifier( loss_function='MultiClass' )
search3_binary_logloss_scale_pos_weight = GridSearchCV( estimator = mdl3_binary_logloss_scale_pos_weight, param_grid=param_grid ,cv=cv, verbose=3, n_jobs=-1  )
search_result3_binary_logloss_scale_pos_weight = search3_binary_logloss_scale_pos_weight.fit( X_train_scaled3, y_train3 )
print("Best: %f using %s" % ( search_result3_binary_logloss_scale_pos_weight.best_score_, search_result3_binary_logloss_scale_pos_weight.best_params_ ) )
print('\n')
dump(search_result3_binary_logloss_scale_pos_weight,PATH + 'CatBoost_Multiclass_ANOMALO3_DataDriven.joblib') 

print('Escolhendo...')
#mdl4_binary_logloss_scale_pos_weight =  LGBMClassifier( class_weight = { 0: DataSet_Train4_weight_backgr.sum(), 1: DataSet_Train4_weight_signal.sum(), 2:DataSet_TrainSM4_weight_signal.sum() } )
mdl4_binary_logloss_scale_pos_weight = CatBoostClassifier( loss_function='MultiClass' )
search4_binary_logloss_scale_pos_weight = GridSearchCV( estimator = mdl4_binary_logloss_scale_pos_weight,param_grid=param_grid ,cv=cv, verbose=3 , n_jobs=-1  )
search_result4_binary_logloss_scale_pos_weight = search4_binary_logloss_scale_pos_weight.fit( X_train_scaled4, y_train4 )
print("Best: %f using %s" % ( search_result4_binary_logloss_scale_pos_weight.best_score_, search_result4_binary_logloss_scale_pos_weight.best_params_ ) )
print('\n')
dump(search_result4_binary_logloss_scale_pos_weight,PATH + 'CatBoost_Multiclass_ANOMALO4_DataDriven.joblib') 


Escolhendo...
Fitting 2 folds for each of 72 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  8.9min
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
from sklearn.preprocessing import label_binarize
def result( X_test, Y_test, X_train, Y_train, model, label, verbose = 1 ):
    print(' ---------------- ' + label + '----------------' )
    scaler = StandardScaler()
    X_scaled_test = scaler.transform(X_test)
    X_scaled_train = scaler.fit_transform(X_train)
    
    predict_proba_test = model.predict_proba(X_scaled_test)  # calculate the probability
    predict_proba_train = model.predict_proba(X_scaled_train)  # calculate the probability

    label_test = label_binarize( Y_test, classes = [ *range(3) ] )
    precision_test = dict()
    recall_test = dict()
    threshs_test = dict()

    for i in range(3):
        precision_test[i], recall_test[i], threshs_test[i] = precision_recall_curve( label_test[:,i], predict_proba_test[:,i] )

    thresholds_test = np.concatenate( [ threshs_test[1], [1] ], axis = 0 )
    bidx_test = np.argmax( precision_test[1] * recall_test[1] )
    best_cut_test = threshs_test[1][bidx_test]
    print('best cut test--> ', best_cut_test)
    y_preds_test = [ np.argmax(line) for line in predict_proba_test > best_cut_test ]

    label_train = label_binarize( Y_train, classes = [ *range(3) ] )
    precision_train = dict()
    recall_train = dict()
    threshs_train = dict()

    for i in range(3):
        precision_train[i], recall_train[i], threshs_train[i] = precision_recall_curve( label_train[:,i], predict_proba_train[:,i] )

    thresholds_train = np.concatenate( [ threshs_train[1], [1] ], axis = 0 )
    bidx_train = np.argmax( precision_train[1] * recall_train[1] )
    best_cut_train = threshs_test[1][bidx_train]
    print('best cut train -->', best_cut_train)
    y_preds_train = [ np.argmax(line) for line in predict_proba_train > best_cut_train ]    

    if verbose == 1:
        print("Purity in test sample     : {:2.2f}%".format(100 * precision_score(Y_test, y_preds_test, average = 'weighted')))
        print("Efficiency in test sample : {:2.2f}%".format(100 * recall_score(Y_test, y_preds_test, average = 'weighted')))
        print( "F1_score in test sample  : {:2.2f}%".format(100 * f1_score(Y_test, y_preds_test, average = 'weighted')))
        print('\n')
        print("Purity in train sample     : {:2.2f}%".format(100 * precision_score(Y_train, y_preds_train, average = 'weighted')))
        print("Efficiency in train sample : {:2.2f}%".format(100 * recall_score(Y_train, y_preds_train, average = 'weighted')))
        print( "F1_score in train sample  : {:2.2f}%".format(100 * f1_score(Y_train, y_preds_train, average = 'weighted')))    

    return predict_proba_test[:,1], best_cut_test        

In [ ]:
predict_proba1, best_cut1 = result(DataSet_Test1, y_test1, DataSet_Train1, y_train1, search_result1_binary_logloss_scale_pos_weight, 'ANOMALO1' )
predict_proba2, best_cut2 = result(DataSet_Test2, y_test2, DataSet_Train2, y_train2, search_result2_binary_logloss_scale_pos_weight, 'ANOMALO2' )
predict_proba3, best_cut3 = result(DataSet_Test3, y_test3, DataSet_Train3, y_train3, search_result3_binary_logloss_scale_pos_weight, 'ANOMALO3' )
predict_proba4, best_cut4 = result(DataSet_Test4, y_test4, DataSet_Train4, y_train4, search_result4_binary_logloss_scale_pos_weight, 'ANOMALO4' )
predict_proba5, best_cut5 = result(DataSet_Test5, y_test5, DataSet_Train5, y_train5, search_result5_binary_logloss_scale_pos_weight, 'ANOMALO5' )
predict_proba6, best_cut6 = result(DataSet_Test6, y_test6, DataSet_Train6, y_train6, search_result6_binary_logloss_scale_pos_weight, 'ANOMALO6' )
predict_proba7, best_cut7 = result(DataSet_Test7, y_test7, DataSet_Train7, y_train7, search_result7_binary_logloss_scale_pos_weight, 'ANOMALO7' )
predict_proba8, best_cut8 = result(DataSet_Test8, y_test8, DataSet_Train8, y_train8, search_result8_binary_logloss_scale_pos_weight, 'ANOMALO8' )

In [ ]:
sys.exit()

In [ ]:
'''search1_binary_logloss_scale_pos_weight = {'subsample': 0.7, 'objective': 'multiclass', 'num_leaves': 50, 'num_class': 3, 'n_estimators': 500, 'min_child_weight': 0.008, 'min_child_samples': 90, 'metric': 'multi_logloss', 'max_depth': 150, 'learning_rate': 0.05, 'feature_pre_filter': 'False', 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}




search2_binary_logloss_scale_pos_weight = {'subsample': 1, 'objective': 'multiclass', 'num_leaves': 100, 'num_class': 3, 'n_estimators': 500, 'min_child_weight': 0.5, 'min_child_samples': 90, 'metric': 'multi_logloss', 'max_depth': 150, 'learning_rate': 0.05, 'feature_pre_filter': 'False', 'colsample_bytree': 1, 'boosting_type': 'gbdt'}




search3_binary_logloss_scale_pos_weight = {'subsample': 0.1, 'objective': 'multiclass', 'num_leaves': 50, 'num_class': 3, 'n_estimators': 500, 'min_child_weight': 0.5, 'min_child_samples': 190, 'metric': 'multi_logloss', 'max_depth': 20, 'learning_rate': 0.008, 'feature_pre_filter': 'False', 'colsample_bytree': 1, 'boosting_type': 'gbdt'}



search4_binary_logloss_scale_pos_weight = {'subsample': 0.7, 'objective': 'multiclass', 'num_leaves': 50, 'num_class': 3, 'n_estimators': 500, 'min_child_weight': 0.5, 'min_child_samples': 190, 'metric': 'multi_logloss', 'max_depth': 400, 'learning_rate': 0.05, 'feature_pre_filter': 'False', 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}


search5_binary_logloss_scale_pos_weight = {'subsample': 0.7, 'objective': 'multiclass', 'num_leaves': 50, 'num_class': 3, 'n_estimators': 500, 'min_child_weight': 0.5, 'min_child_samples': 20, 'metric': 'multi_logloss', 'max_depth': 1, 'learning_rate': 0.1, 'feature_pre_filter': 'False', 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}



search6_binary_logloss_scale_pos_weight = {'subsample': 1, 'objective': 'multiclass', 'num_leaves': 50, 'num_class': 3, 'n_estimators': 500, 'min_child_weight': 0.5, 'min_child_samples': 190, 'metric': 'multi_logloss', 'max_depth': 400, 'learning_rate': 0.008, 'feature_pre_filter': 'False', 'colsample_bytree': 1, 'boosting_type': 'gbdt'}


search7_binary_logloss_scale_pos_weight = {'subsample': 0.7, 'objective': 'multiclass', 'num_leaves': 50, 'num_class': 3, 'n_estimators': 500, 'min_child_weight': 0.001, 'min_child_samples': 190, 'metric': 'multi_logloss', 'max_depth': 150, 'learning_rate': 0.05, 'feature_pre_filter': 'False', 'colsample_bytree': 1, 'boosting_type': 'gbdt'}



search8_binary_logloss_scale_pos_weight = {'subsample': 0.7, 'objective': 'multiclass', 'num_leaves': 50, 'num_class': 3, 'n_estimators': 500, 'min_child_weight': 0.008, 'min_child_samples': 90, 'metric': 'multi_logloss', 'max_depth': 150, 'learning_rate': 0.05, 'feature_pre_filter': 'False', 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}

'''

In [ ]:
print('Treinando os modelos...\n')

bst8_binary_logloss_scale_pos_weight = lgb.train(search_result8_binary_logloss_scale_pos_weight.best_params_  , train_LGB8)
bst7_binary_logloss_scale_pos_weight = lgb.train(search_result7_binary_logloss_scale_pos_weight.best_params_  , train_LGB7)
bst6_binary_logloss_scale_pos_weight = lgb.train(search_result6_binary_logloss_scale_pos_weight.best_params_  , train_LGB6)
bst5_binary_logloss_scale_pos_weight = lgb.train(search_result5_binary_logloss_scale_pos_weight.best_params_ , train_LGB5)

bst1_binary_logloss_scale_pos_weight = lgb.train(search_result1_binary_logloss_scale_pos_weight.best_params_ , train_LGB1)
bst2_binary_logloss_scale_pos_weight = lgb.train(search_result2_binary_logloss_scale_pos_weight.best_params_  , train_LGB2)
bst3_binary_logloss_scale_pos_weight = lgb.train(search_result3_binary_logloss_scale_pos_weight.best_params_ , train_LGB3)
bst4_binary_logloss_scale_pos_weight = lgb.train(search_result4_binary_logloss_scale_pos_weight.best_params_  , train_LGB4)

In [ ]:
'''from joblib import dump, load
dump( bst8_binary_logloss_scale_pos_weight, "bst8_binary_logloss_scale_pos_weight.joblib" )
dump( bst7_binary_logloss_scale_pos_weight, "bst7_binary_logloss_scale_pos_weight.joblib" )
dump( bst6_binary_logloss_scale_pos_weight, "bst6_binary_logloss_scale_pos_weight.joblib" )
dump( bst5_binary_logloss_scale_pos_weight, "bst5_binary_logloss_scale_pos_weight.joblib" )
dump( bst4_binary_logloss_scale_pos_weight, "bst4_binary_logloss_scale_pos_weight.joblib" )
dump( bst3_binary_logloss_scale_pos_weight, "bst3_binary_logloss_scale_pos_weight.joblib" )
dump( bst2_binary_logloss_scale_pos_weight, "bst2_binary_logloss_scale_pos_weight.joblib" )
dump( bst1_binary_logloss_scale_pos_weight, "bst1_binary_logloss_scale_pos_weight.joblib" )'''

In [ ]:
predict_proba_LGBM8_binary_logloss_scale_pos_weight = bst8_binary_logloss_scale_pos_weight.predict(DataSet_Test8)
print('Predict Proba Anomalo 8 --> \n', predict_proba_LGBM8_binary_logloss_scale_pos_weight)

predict_proba_LGBM7_binary_logloss_scale_pos_weight = bst7_binary_logloss_scale_pos_weight.predict(DataSet_Test7)
print('Predict Proba Anomalo 7 --> \n', predict_proba_LGBM7_binary_logloss_scale_pos_weight)

predict_proba_LGBM6_binary_logloss_scale_pos_weight = bst6_binary_logloss_scale_pos_weight.predict(DataSet_Test6)
print('Predict Proba Anomalo 6 --> \n', predict_proba_LGBM6_binary_logloss_scale_pos_weight)

predict_proba_LGBM5_binary_logloss_scale_pos_weight = bst5_binary_logloss_scale_pos_weight.predict(DataSet_Test5)
print('Predict Proba Anomalo 5 --> \n', predict_proba_LGBM5_binary_logloss_scale_pos_weight)

predict_proba_LGBM1_binary_logloss_scale_pos_weight = bst1_binary_logloss_scale_pos_weight.predict(DataSet_Test1)
print('Predict Proba Anomalo 1 --> \n', predict_proba_LGBM1_binary_logloss_scale_pos_weight)

predict_proba_LGBM2_binary_logloss_scale_pos_weight = bst2_binary_logloss_scale_pos_weight.predict(DataSet_Test2)
print('Predict Proba Anomalo 2 --> \n', predict_proba_LGBM2_binary_logloss_scale_pos_weight)

predict_proba_LGBM3_binary_logloss_scale_pos_weight = bst3_binary_logloss_scale_pos_weight.predict(DataSet_Test3)
print('Predict Proba Anomalo 3 --> \n', predict_proba_LGBM3_binary_logloss_scale_pos_weight)

predict_proba_LGBM4_binary_logloss_scale_pos_weight = bst4_binary_logloss_scale_pos_weight.predict(DataSet_Test4)
print('Predict Proba Anomalo 4 --> \n', predict_proba_LGBM4_binary_logloss_scale_pos_weight)

In [ ]:
from sklearn.preprocessing import label_binarize

In [ ]:
def plot_precision_recall_curve( label_test, predicit, title, nome ):
    label_test = label_binarize( label_test, classes = [*range(3) ] )
    precision = dict()
    recall = dict()
    threshs = dict()
    fontsize = 18

    fig, axes = plt.subplots( 1, 2, figsize=(19,10) )

    for i in range(3):
        precision[i], recall[i], threshs[i] = precision_recall_curve( label_test[:,i], predicit[:, i])
        axes[0].plot( recall[i], precision[i], lw = 2, label = ' class {}'.format(i) )
    axes[0].set_xlabel( "Recall", fontsize = fontsize )
    axes[0].set_ylabel( "Precision", fontsize = fontsize )
    axes[0].legend( loc = "best", fontsize = 14 )
    axes[0].set_title( title)

    thresholds = np.concatenate( [ threshs[1], [1] ], axis = 0 )
    bidx = np.argmax( precision[1] * recall[1] )
    best_cut = threshs[1][bidx] 
    print( '\n',' Best Cut ', best_cut,'\n' )

    axes[1].plot( thresholds , precision[1] * recall[1],  color = 'blue' )
    axes[1].plot( [ best_cut , best_cut ] , [-0.1,(precision[1] * recall[1]).max()*2] ,"-.r",label='Best Cut : {:2.5f}'.format(best_cut) )
    axes[1].plot( [-0.1,1.1] , [ ( precision[1]*recall[1] ).max(),( precision[1]*recall[1] ).max() ], ":g",label=r'Precision $\times$ Recall : {:2.5f}'.format( (precision[1]*recall[1]).max() ) )
    axes[1].set_ylabel( r'Precision $\times$ Recall', fontsize = fontsize )
    axes[1].set_xlabel(    'Thresholds', fontsize = fontsize    )
    axes[1].set_ylim( -0.1 , 1 )
    axes[1].set_xlim( -0.01 , 1 )
    axes[1].set_title( title )
    axes[1].legend( loc = "best", fontsize = 14)  
    #plt.savefig('/afs/cern.ch/user/m/matheus/WWCEP_multiclass_ML/output_multiclass/' + title + '.jpg')
    #plt.close()


In [ ]:
def best_cut( label_test, predicit ):
    label_test = label_binarize( label_test, classes = [ *range(3) ] )
    precision = dict()
    recall = dict()
    threshs = dict()

    for i in range(3):
        precision[i], recall[i], threshs[i] = precision_recall_curve( label_test[:,i], predicit[:, i] )

    thresholds = np.concatenate( [ threshs[1], [1] ], axis = 0 )
    bidx = np.argmax( precision[1] * recall[1] )
    return threshs[1][bidx]  

In [ ]:
plot_precision_recall_curve(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', 'Precision_Recal_Anomalo8' )

In [ ]:
plot_precision_recall_curve(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', 'Precision_Recal_Anomalo7' )

In [ ]:
plot_precision_recall_curve(y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 1.0 \times 10^{-6} $', 'Precision_Recal_Anomalo6' )

In [ ]:
plot_precision_recall_curve(y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 0.5 \times 10^{-6} $', 'Precision_Recal_Anomalo5' )

In [ ]:
plot_precision_recall_curve(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 2.0 \times 10^{-5} $', 'Precision_Recal_Anomalo1' )

In [ ]:
plot_precision_recall_curve(y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', 'Precision_Recal_Anomalo2' )

In [ ]:
plot_precision_recall_curve(y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 8.0 \times 10^{-6} $', 'Precision_Recal_Anomalo4' )

In [ ]:
plot_precision_recall_curve(y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', 'Precision_Recal_Anomalo3' )

In [ ]:
select_columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass', 'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'Yww', 'xi1', 'xi2', 'Mx', 'Yx', 'Mww/Mx', 'Yww_Yx']

In [ ]:
#argmax() method 
y_pred_1 = [np.argmax(line) for line in predict_proba_LGBM1_binary_logloss_scale_pos_weight > best_cut( y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight )]
y_pred_2 = [np.argmax(line) for line in predict_proba_LGBM2_binary_logloss_scale_pos_weight > best_cut( y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight )]
y_pred_3 = [np.argmax(line) for line in predict_proba_LGBM3_binary_logloss_scale_pos_weight > best_cut( y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight )]
y_pred_4 = [np.argmax(line) for line in predict_proba_LGBM4_binary_logloss_scale_pos_weight > best_cut( y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight )]
y_pred_5 = [np.argmax(line) for line in predict_proba_LGBM5_binary_logloss_scale_pos_weight > best_cut( y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight )]
y_pred_6 = [np.argmax(line) for line in predict_proba_LGBM6_binary_logloss_scale_pos_weight > best_cut( y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight )]
y_pred_7 = [np.argmax(line) for line in predict_proba_LGBM7_binary_logloss_scale_pos_weight > best_cut( y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight )]
y_pred_8 = [np.argmax(line) for line in predict_proba_LGBM8_binary_logloss_scale_pos_weight > best_cut( y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight )]

from sklearn.metrics import recall_score,classification_report, precision_score

In [ ]:
fig, ax = plt.subplots( 2, 2, figsize=(24,15) )
predict_dados = bst4_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,0].hist( predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test4_weight_backgr/test_size )
ax[0,0].hist( predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=8.0 \times 10^{-6}$', weights = DataSet_Test4_weight_signal/test_size, color = 'green' )
ax[0,0].hist( predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM4_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,0].plot( [ best_cut(y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight),best_cut( y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut4 = predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 0 ][:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut4 = predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 1 ][:,1] > best_cut(y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight )
n_events_SM_after_cut4 = predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 2 ][:,1] > best_cut(y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut4 = predict_dados[ predict_dados >= best_cut( y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ]

print(" --------------- Anomalo 4 --------------- ")
print('Numero de eventos de background depois do corte -->', DataSet_Test4_weight_backgr[ n_events_back_after_cut4 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test4_weight_signal[ n_events_signal_after_cut4].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM4_weight_signal[ n_events_SM_after_cut4 ].sum() / test_size )
print('Recall Score',recall_score(y_test4,y_pred_4, average = 'weighted') )
print('Precision Score',precision_score(y_test4,y_pred_4, average = 'weighted') )

ax[0,0].text(best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,0].set_ylabel( "Events (Log Scale)" )
ax[0,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,0].legend( loc = "upper right", fontsize = 16 )
ax[0,0].set_yscale( 'log' )
ax[0,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,0] )


predict_dados = bst1_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,1].hist( predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test1_weight_backgr/test_size )
ax[0,1].hist( predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=20 \times 10^{-6}$', weights = DataSet_Test1_weight_signal/test_size, color = 'green' )
ax[0,1].hist( predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM1_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,1].plot( [ best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight),best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut1 = predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 0 ][:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut1 = predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 1 ][:,1] > best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight )
n_events_SM_after_cut1 = predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 2 ][:,1] > best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut1 = predict_dados[ predict_dados >= best_cut( y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ]

print(' --------------- Anomalo 1 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test1_weight_backgr[ n_events_back_after_cut1 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test1_weight_signal[ n_events_signal_after_cut1].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM1_weight_signal[ n_events_SM_after_cut1 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut1 ) )
print('Recall Score',recall_score(y_test1,y_pred_1, average = 'weighted') )
print('Precision Score',precision_score(y_test1,y_pred_1, average = 'weighted') )

ax[0,1].text(best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,1].legend( loc = "upper right", fontsize = 16 )
ax[0,1].set_yscale( 'log' )
ax[0,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,1] )

predict_dados = bst3_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,0].hist( predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test3_weight_backgr/test_size )
ax[1,0].hist( predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', weights = DataSet_Test3_weight_signal/test_size, color = 'green' )
ax[1,0].hist( predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM3_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,0].plot( [ best_cut(y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight),best_cut(y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut3 = predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 0 ][:,1] >= best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut3 = predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 1 ][:,1] >= best_cut(y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight )
n_events_SM_after_cut3 = predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 2 ][:,1] >= best_cut(y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut3 = predict_dados[ predict_dados >= best_cut( y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ]

print(' --------------- Anomalo 3 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test3_weight_backgr[ n_events_back_after_cut3 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test3_weight_signal[ n_events_signal_after_cut3].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM3_weight_signal[ n_events_SM_after_cut3 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut3 ) )
print('Recall Score',recall_score(y_test3,y_pred_3, average = 'weighted') )
print('Precision Score',precision_score(y_test3,y_pred_3, average = 'weighted') )

ax[1,0].text(best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,0].legend( loc = "upper right", fontsize = 16 )
ax[1,0].set_yscale( 'log' )
ax[1,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,0] )

predict_dados = bst2_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,1].hist( predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test2_weight_backgr/test_size )
ax[1,1].hist( predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', weights = DataSet_Test2_weight_signal/test_size, color = 'green' )
ax[1,1].hist( predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standar Model', weights = DataSet_TestSM2_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,1].plot( [ best_cut(y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight),best_cut(y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut2 = predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 0 ][:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut2 = predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 1 ][:,1] > best_cut(y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight )
n_events_SM_after_cut2 = predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 2 ][:,1] > best_cut(y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut2 = predict_dados[ predict_dados > best_cut( y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ]

print(' --------------- Anomalo 2 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test2_weight_backgr[ n_events_back_after_cut2 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test2_weight_signal[ n_events_signal_after_cut2 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM2_weight_signal[ n_events_SM_after_cut2 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut2 ) )
print('Recall Score',recall_score(y_test2,y_pred_2, average = 'weighted') )
print('Precision Score',precision_score(y_test2,y_pred_2, average = 'weighted') )

ax[1,1].set_xlabel( "Probability" )
ax[1,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,1].legend( loc = "upper right", fontsize = 16 )
ax[1,1].set_yscale( 'log' )
ax[1,1].text(best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,1] )

In [ ]:
fig, ax = plt.subplots( 2, 2, figsize=(24,15) )
predict_dados = bst5_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,0].hist( predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test5_weight_backgr/test_size )
ax[0,0].hist( predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=0.5 \times 10^{-6}$', weights = DataSet_Test5_weight_signal/test_size, color = 'green' )
ax[0,0].hist( predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM5_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,0].plot( [ best_cut(y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight),best_cut( y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut5 = predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 0 ][:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut5 = predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 1 ][:,1] > best_cut(y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight )
n_events_SM_after_cut5 = predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 2 ][:,1] > best_cut(y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut5 = predict_dados[ predict_dados >= best_cut( y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ]

print("Anomalo 5 ")
print('Numero de eventos de background depois do corte -->', DataSet_Test5_weight_backgr[ n_events_back_after_cut5 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test5_weight_signal[ n_events_signal_after_cut5 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM5_weight_signal[ n_events_SM_after_cut5 ].sum() / test_size )
print('Recall Score',recall_score(y_test5,y_pred_5, average = 'weighted') )
print('Precision Score',precision_score(y_test5,y_pred_5, average = 'weighted') )

ax[0,0].text(best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,0].set_ylabel( "Events (Log Scale)" )
ax[0,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,0].legend( loc = "upper right", fontsize = 16 )
ax[0,0].set_yscale( 'log' )
ax[0,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,0] )


predict_dados = bst6_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,1].hist( predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test6_weight_backgr/test_size )
ax[0,1].hist( predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=1.0 \times 10^{-5}$', weights = DataSet_Test6_weight_signal/test_size, color = 'green' )
ax[0,1].hist( predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM6_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,1].plot( [ best_cut(y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight),best_cut(y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut6 = predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 0 ][:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut6 = predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 1 ][:,1] > best_cut(y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight )
n_events_SM_after_cut6 = predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 2 ][:,1] > best_cut(y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut6 = predict_dados[ predict_dados >= best_cut( y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ]

print('Anomalo 6')
print('Numero de eventos de background depois do corte -->', DataSet_Test6_weight_backgr[ n_events_back_after_cut6 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test6_weight_signal[ n_events_signal_after_cut6].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM6_weight_signal[ n_events_SM_after_cut6 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut6 ) )
print('Recall Score',recall_score(y_test6,y_pred_6, average = 'weighted') )
print('Precision Score',precision_score(y_test6,y_pred_6, average = 'weighted') ) 

ax[0,1].text(best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,1].legend( loc = "upper right", fontsize = 16 )
ax[0,1].set_yscale( 'log' )
ax[0,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,1] )

predict_dados = bst7_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,0].hist( predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test7_weight_backgr/test_size )
ax[1,0].hist( predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', weights = DataSet_Test7_weight_signal/test_size, color = 'green' )
ax[1,0].hist( predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM7_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,0].plot( [ best_cut(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight),best_cut(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut7 = predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 0 ][:,1] >= best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut7 = predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 1 ][:,1] >= best_cut(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight )
n_events_SM_after_cut7 = predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 2 ][:,1] >= best_cut(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut7 = predict_dados[ predict_dados >= best_cut( y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ]

print('Anomalo 7')
print('Numero de eventos de background depois do corte -->', DataSet_Test7_weight_backgr[ n_events_back_after_cut7 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test7_weight_signal[ n_events_signal_after_cut7].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM7_weight_signal[ n_events_SM_after_cut7 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut7 ) )
print('Recall Score',recall_score(y_test7,y_pred_7, average = 'weighted') )
print('Precision Score',precision_score(y_test7,y_pred_7, average = 'weighted') )

ax[1,0].text(best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,0].legend( loc = "upper right", fontsize = 16 )
ax[1,0].set_yscale( 'log' )
ax[1,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,0] )

predict_dados = bst8_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,1].hist( predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test8_weight_backgr/test_size )
ax[1,1].hist( predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', weights = DataSet_Test8_weight_signal/test_size, color = 'green' )
ax[1,1].hist( predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standar Model', weights = DataSet_TestSM8_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,1].plot( [ best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight),best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut8 = predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 0 ][:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut8 = predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 1 ][:,1] > best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight )
n_events_SM_after_cut8 = predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 2 ][:,1] > best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut8 = predict_dados[ predict_dados > best_cut( y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ]

print('Anomalo 8')
print('Numero de eventos de background depois do corte -->', DataSet_Test8_weight_backgr[ n_events_back_after_cut8 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test8_weight_signal[ n_events_signal_after_cut8 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM8_weight_signal[ n_events_SM_after_cut8 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut8 ) )
print('Recall Score',recall_score(y_test8,y_pred_8, average = 'weighted') )
print('Precision Score',precision_score(y_test8,y_pred_8, average = 'weighted') )

ax[1,1].set_xlabel( "Probability" )
ax[1,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,1].legend( loc = "upper right", fontsize = 16 )
ax[1,1].set_yscale( 'log' )
ax[1,1].text(best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,1] )

In [ ]:
fig, ax = plt.subplots( 2, 2, figsize=(24,15) )
predict_dados = model_ANOMALO4.predict(data_set_dados_multirp)[:,1]
bins = 25

hbgt_lgb = ax[0,0].hist( predict_ANOMALO4[ y_test4 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Driven Background', weights = DataSet_Test4_weight_backgr/test_size )
ax[0,0].hist( predict_ANOMALO4[ y_test4 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=8.0 \times 10^{-6}$', weights = DataSet_Test4_weight_signal/test_size, color = 'green' )
ax[0,0].hist( predict_ANOMALO4[ y_test4 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM4_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,0].plot( [ best_cut_ANOMALO4,best_cut_ANOMALO4],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO4 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO4 ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut4 = predict_ANOMALO4[ y_test4 == 0 ][:,1] > best_cut_ANOMALO4 
n_events_signal_after_cut4 = predict_ANOMALO4[ y_test4 == 1 ][:,1] > best_cut_ANOMALO4
n_events_SM_after_cut4 = predict_ANOMALO4[ y_test4 == 2 ][:,1] > best_cut_ANOMALO4
n_eventos_Data_after_cut4 = predict_dados[ predict_dados >= best_cut_ANOMALO4]

print(" --------------- Anomalo 4 --------------- ")
print('Numero de eventos de background depois do corte -->', DataSet_Test4_weight_backgr[ n_events_back_after_cut4 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test4_weight_signal[ n_events_signal_after_cut4].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM4_weight_signal[ n_events_SM_after_cut4 ].sum() / test_size )


ax[0,0].text(best_cut_ANOMALO4+0.05 , 0.1, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,0].set_ylabel( "Events (Log Scale)" )
ax[0,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,0].legend( loc = "upper right", fontsize = 16 )
ax[0,0].set_yscale( 'log' )
ax[0,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,0] )


predict_dados = model_ANOMALO1.predict(data_set_dados_multirp)[:,1]
bins = 25

hbgt_lgb = ax[0,1].hist( predict_ANOMALO1[ y_test1 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Drive Background', weights = DataSet_Test1_weight_backgr/test_size )
ax[0,1].hist( predict_ANOMALO1[ y_test1 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=20 \times 10^{-6}$', weights = DataSet_Test1_weight_signal/test_size, color = 'green' )
ax[0,1].hist( predict_ANOMALO1[ y_test1 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM1_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,1].plot( [ best_cut_ANOMALO1,best_cut_ANOMALO1],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO1 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO1] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut1 = predict_ANOMALO1[ y_test1 == 0 ][:,1] > best_cut_ANOMALO1
n_events_signal_after_cut1 = predict_ANOMALO1[ y_test1 == 1 ][:,1] > best_cut_ANOMALO1
n_events_SM_after_cut1 = predict_ANOMALO1[ y_test1 == 2 ][:,1] > best_cut_ANOMALO1
n_eventos_Data_after_cut1 = predict_dados[ predict_dados >= best_cut_ANOMALO1 ]

print(' --------------- Anomalo 1 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test1_weight_backgr[ n_events_back_after_cut1 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test1_weight_signal[ n_events_signal_after_cut1].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM1_weight_signal[ n_events_SM_after_cut1 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut1 ) )


ax[0,1].text(best_cut_ANOMALO1 + 0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,1].legend( loc = "upper right", fontsize = 16 )
ax[0,1].set_yscale( 'log' )
ax[0,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,1] )

predict_dados = model_ANOMALO3.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,0].hist( predict_ANOMALO3[ y_test3 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Driven Background', weights = DataSet_Test3_weight_backgr/test_size )
ax[1,0].hist( predict_ANOMALO3[ y_test3 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', weights = DataSet_Test3_weight_signal/test_size, color = 'green' )
ax[1,0].hist( predict_ANOMALO3[ y_test3 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM3_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,0].plot( [ best_cut_ANOMALO3,best_cut_ANOMALO3],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO3 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO3 ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Dados-2016', color = 'black' )

n_events_back_after_cut3 = predict_ANOMALO3[ y_test3 == 0 ][:,1] >= best_cut_ANOMALO3
n_events_signal_after_cut3 = predict_ANOMALO3[ y_test3 == 1 ][:,1] >= best_cut_ANOMALO3
n_events_SM_after_cut3 = predict_ANOMALO3[ y_test3 == 2 ][:,1] >= best_cut_ANOMALO3
n_eventos_Data_after_cut3 = predict_dados[ predict_dados >= best_cut_ANOMALO3 ]

print(' --------------- Anomalo 3 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test3_weight_backgr[ n_events_back_after_cut3 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test3_weight_signal[ n_events_signal_after_cut3].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM3_weight_signal[ n_events_SM_after_cut3 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut3 ) )


ax[1,0].text(best_cut_ANOMALO3+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,0].legend( loc = "upper right", fontsize = 16 )
ax[1,0].set_yscale( 'log' )
ax[1,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,0] )

predict_dados = model_ANOMALO2.predict(data_set_dados_multirp)[:,1]
bins = 25

hbgt_lgb = ax[1,1].hist( predict_ANOMALO2[ y_test2 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Driven Background', weights = DataSet_Test2_weight_backgr/test_size )
ax[1,1].hist( predict_ANOMALO2[ y_test2 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', weights = DataSet_Test2_weight_signal/test_size, color = 'green' )
ax[1,1].hist( predict_ANOMALO2[ y_test2 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM2_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,1].plot( [ best_cut(y_test2,predict_ANOMALO2),best_cut_ANOMALO2],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO2 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO2 ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut2 = predict_ANOMALO2[ y_test2 == 0 ][:,1] > best_cut_ANOMALO2 
n_events_signal_after_cut2 = predict_ANOMALO2[ y_test2 == 1 ][:,1] > best_cut_ANOMALO2
n_events_SM_after_cut2 = predict_ANOMALO2[ y_test2 == 2 ][:,1] > best_cut_ANOMALO2
n_eventos_Data_after_cut2 = predict_dados[ predict_dados > best_cut_ANOMALO2 ]

print(' --------------- Anomalo 2 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test2_weight_backgr[ n_events_back_after_cut2 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test2_weight_signal[ n_events_signal_after_cut2 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM2_weight_signal[ n_events_SM_after_cut2 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut2 ) )

ax[1,1].set_xlabel( "Probability" )
ax[1,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,1].legend( loc = "upper right", fontsize = 16 )
ax[1,1].set_yscale( 'log' )
ax[1,1].text(best_cut_ANOMALO2+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,1] )

# **Aplicando o resultado do treinamento das constantes maiores nas menores**

In [ ]:
predict_proba_TraningAnomalo8_overAnomalo5 = bst8_binary_logloss_scale_pos_weight.predict(DataSet_Test5)
print('Predict Proba Anomalo 8 --> \n', predict_proba_TraningAnomalo8_overAnomalo5)

predict_proba_TraningAnomalo8_overAnomalo6 = bst8_binary_logloss_scale_pos_weight.predict(DataSet_Test6)
print('Predict Proba Anomalo 8 --> \n', predict_proba_TraningAnomalo8_overAnomalo6)

predict_proba_TraningAnomalo8_overAnomalo7 = bst8_binary_logloss_scale_pos_weight.predict(DataSet_Test7)
print('Predict Proba Anomalo 8 --> \n', predict_proba_TraningAnomalo8_overAnomalo7)


predict_proba_TraningAnomalo1_overAnomalo2 = bst1_binary_logloss_scale_pos_weight.predict(DataSet_Test2)
print('Predict Proba Anomalo 8 --> \n', predict_proba_TraningAnomalo1_overAnomalo2)

predict_proba_TraningAnomalo1_overAnomalo3 = bst1_binary_logloss_scale_pos_weight.predict(DataSet_Test3)
print('Predict Proba Anomalo 8 --> \n', predict_proba_TraningAnomalo1_overAnomalo3)

predict_proba_TraningAnomalo1_overAnomalo4 = bst1_binary_logloss_scale_pos_weight.predict(DataSet_Test4)
print('Predict Proba Anomalo 8 --> \n', predict_proba_TraningAnomalo1_overAnomalo4)

In [ ]:
fig, ax = plt.subplots( 2, 2, figsize=(24,15) )
predict_dados = bst8_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,0].hist( predict_proba_TraningAnomalo8_overAnomalo5[ y_test5 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'gold', label = 'Background', weights = DataSet_Test5_weight_backgr/test_size )
ax[0,0].hist( predict_proba_TraningAnomalo8_overAnomalo5[ y_test5 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=0.5 \times 10^{-6}$', weights = DataSet_Test5_weight_signal/test_size, color = 'green' )
ax[0,0].hist( predict_proba_TraningAnomalo8_overAnomalo5[ y_test5 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM5_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,0].plot( [ best_cut(y_test5,predict_proba_TraningAnomalo8_overAnomalo5),best_cut( y_test5,predict_proba_TraningAnomalo8_overAnomalo5)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test5, predict_proba_TraningAnomalo8_overAnomalo5) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test5,predict_proba_TraningAnomalo8_overAnomalo5 ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut5 = predict_proba_TraningAnomalo8_overAnomalo5[ y_test5 == 0 ][:,1] > best_cut( y_test5, predict_proba_TraningAnomalo8_overAnomalo5 ) 
n_events_signal_after_cut5 = predict_proba_TraningAnomalo8_overAnomalo5[ y_test5 == 1 ][:,1] > best_cut(y_test5,predict_proba_TraningAnomalo8_overAnomalo5 )
n_events_SM_after_cut5 = predict_proba_TraningAnomalo8_overAnomalo5[ y_test5 == 2 ][:,1] > best_cut(y_test5,predict_proba_TraningAnomalo8_overAnomalo5 )
n_eventos_Data_after_cut5 = predict_dados[ predict_dados >= best_cut( y_test5,predict_proba_TraningAnomalo8_overAnomalo5 ) ]

print("Anomalo 5 ")
print('Numero de eventos de background depois do corte -->', DataSet_Test5_weight_backgr[ n_events_back_after_cut5 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test5_weight_signal[ n_events_signal_after_cut5 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM5_weight_signal[ n_events_SM_after_cut5 ].sum() / test_size )
print('Recall Score',recall_score(y_test5,y_pred_5, average = 'weighted') )
print('Precision Score',precision_score(y_test5,y_pred_5, average = 'weighted') )

ax[0,0].text(best_cut( y_test5, predict_proba_TraningAnomalo8_overAnomalo5 )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,0].set_ylabel( "Events (Log Scale)" )
ax[0,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,0].legend( loc = "upper right", fontsize = 16 )
ax[0,0].set_yscale( 'log' )
ax[0,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,0] )


predict_dados = bst8_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,1].hist( predict_proba_TraningAnomalo8_overAnomalo6[ y_test6 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'gold', label = 'Background', weights = DataSet_Test6_weight_backgr/test_size )
ax[0,1].hist( predict_proba_TraningAnomalo8_overAnomalo6[ y_test6 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=1.0 \times 10^{-5}$', weights = DataSet_Test6_weight_signal/test_size, color = 'green' )
ax[0,1].hist( predict_proba_TraningAnomalo8_overAnomalo6[ y_test6 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM6_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,1].plot( [ best_cut(y_test6,predict_proba_TraningAnomalo8_overAnomalo6),best_cut(y_test6,predict_proba_TraningAnomalo8_overAnomalo6)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test6, predict_proba_TraningAnomalo8_overAnomalo6) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test6,predict_proba_TraningAnomalo8_overAnomalo6 ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut6 = predict_proba_TraningAnomalo8_overAnomalo6[ y_test6 == 0 ][:,1] > best_cut( y_test6, predict_proba_TraningAnomalo8_overAnomalo6 ) 
n_events_signal_after_cut6 = predict_proba_TraningAnomalo8_overAnomalo6[ y_test6 == 1 ][:,1] > best_cut(y_test6,predict_proba_TraningAnomalo8_overAnomalo6 )
n_events_SM_after_cut6 = predict_proba_TraningAnomalo8_overAnomalo6[ y_test6 == 2 ][:,1] > best_cut(y_test6,predict_proba_TraningAnomalo8_overAnomalo6 )
n_eventos_Data_after_cut6 = predict_dados[ predict_dados >= best_cut( y_test6,predict_proba_TraningAnomalo8_overAnomalo6 ) ]

print('Anomalo 6')
print('Numero de eventos de background depois do corte -->', DataSet_Test6_weight_backgr[ n_events_back_after_cut6 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test6_weight_signal[ n_events_signal_after_cut6].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM6_weight_signal[ n_events_SM_after_cut6 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut6 ) )
print('Recall Score',recall_score(y_test6,y_pred_6, average = 'weighted') )
print('Precision Score',precision_score(y_test6,y_pred_6, average = 'weighted') ) 

ax[0,1].text(best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,1].legend( loc = "upper right", fontsize = 16 )
ax[0,1].set_yscale( 'log' )
ax[0,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,1] )

predict_dados = bst8_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,0].hist( predict_proba_TraningAnomalo8_overAnomalo7[ y_test7 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'gold', label = 'Background', weights = DataSet_Test7_weight_backgr/test_size )
ax[1,0].hist( predict_proba_TraningAnomalo8_overAnomalo7[ y_test7 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', weights = DataSet_Test7_weight_signal/test_size, color = 'green' )
ax[1,0].hist( predict_proba_TraningAnomalo8_overAnomalo7[ y_test7 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM7_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,0].plot( [ best_cut(y_test7,predict_proba_TraningAnomalo8_overAnomalo7),best_cut(y_test7,predict_proba_TraningAnomalo8_overAnomalo7)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test7, predict_proba_TraningAnomalo8_overAnomalo7) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test7,predict_proba_TraningAnomalo8_overAnomalo7 ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut7 = predict_proba_TraningAnomalo8_overAnomalo7[ y_test7 == 0 ][:,1] >= best_cut( y_test7, predict_proba_TraningAnomalo8_overAnomalo7 ) 
n_events_signal_after_cut7 = predict_proba_TraningAnomalo8_overAnomalo7[ y_test7 == 1 ][:,1] >= best_cut(y_test7,predict_proba_TraningAnomalo8_overAnomalo7 )
n_events_SM_after_cut7 = predict_proba_TraningAnomalo8_overAnomalo7[ y_test7 == 2 ][:,1] >= best_cut(y_test7,predict_proba_TraningAnomalo8_overAnomalo7 )
n_eventos_Data_after_cut7 = predict_dados[ predict_dados >= best_cut( y_test7,predict_proba_TraningAnomalo8_overAnomalo7 ) ]

print('Anomalo 7')
print('Numero de eventos de background depois do corte -->', DataSet_Test7_weight_backgr[ n_events_back_after_cut7 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test7_weight_signal[ n_events_signal_after_cut7].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM7_weight_signal[ n_events_SM_after_cut7 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut7 ) )
print('Recall Score',recall_score(y_test7,y_pred_7, average = 'weighted') )
print('Precision Score',precision_score(y_test7,y_pred_7, average = 'weighted') )

ax[1,0].text(best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,0].legend( loc = "upper right", fontsize = 16 )
ax[1,0].set_yscale( 'log' )
ax[1,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,0] )

predict_dados = bst8_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,1].hist( predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test8_weight_backgr/test_size )
ax[1,1].hist( predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', weights = DataSet_Test8_weight_signal/test_size, color = 'green' )
ax[1,1].hist( predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standar Model', weights = DataSet_TestSM8_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,1].plot( [ best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight),best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut8 = predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 0 ][:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut8 = predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 1 ][:,1] > best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight )
n_events_SM_after_cut8 = predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 2 ][:,1] > best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut8 = predict_dados[ predict_dados > best_cut( y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ]

print('Anomalo 8')
print('Numero de eventos de background depois do corte -->', DataSet_Test8_weight_backgr[ n_events_back_after_cut8 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test8_weight_signal[ n_events_signal_after_cut8 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM8_weight_signal[ n_events_SM_after_cut8 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut8 ) )
print('Recall Score',recall_score(y_test8,y_pred_8, average = 'weighted') )
print('Precision Score',precision_score(y_test8,y_pred_8, average = 'weighted') )

ax[1,1].set_xlabel( "Probability" )
ax[1,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,1].legend( loc = "upper right", fontsize = 16 )
ax[1,1].set_yscale( 'log' )
ax[1,1].text(best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,1] )

In [ ]:
fig, ax = plt.subplots( 2, 2, figsize=(24,15) )
predict_dados = bst1_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,0].hist( predict_proba_TraningAnomalo1_overAnomalo4[ y_test4 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'gold', label = 'Background', weights = DataSet_Test4_weight_backgr/test_size )
ax[0,0].hist( predict_proba_TraningAnomalo1_overAnomalo4[ y_test4 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=8.0 \times 10^{-6}$', weights = DataSet_Test4_weight_signal/test_size, color = 'green' )
ax[0,0].hist( predict_proba_TraningAnomalo1_overAnomalo4[ y_test4 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM4_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,0].plot( [ best_cut(y_test4,predict_proba_TraningAnomalo1_overAnomalo4),best_cut( y_test4,predict_proba_TraningAnomalo1_overAnomalo4)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test4, predict_proba_TraningAnomalo1_overAnomalo4) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test4,predict_proba_TraningAnomalo1_overAnomalo4 ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut4 = predict_proba_TraningAnomalo1_overAnomalo4[ y_test4 == 0 ][:,1] > best_cut( y_test4, predict_proba_TraningAnomalo1_overAnomalo4 ) 
n_events_signal_after_cut4 = predict_proba_TraningAnomalo1_overAnomalo4[ y_test4 == 1 ][:,1] > best_cut(y_test4,predict_proba_TraningAnomalo1_overAnomalo4 )
n_events_SM_after_cut4 = predict_proba_TraningAnomalo1_overAnomalo4[ y_test4 == 2 ][:,1] > best_cut(y_test4,predict_proba_TraningAnomalo1_overAnomalo4 )
n_eventos_Data_after_cut4 = predict_dados[ predict_dados >= best_cut( y_test4,predict_proba_TraningAnomalo1_overAnomalo4 ) ]

print(" --------------- Anomalo 4 --------------- ")
print('Numero de eventos de background depois do corte -->', DataSet_Test4_weight_backgr[ n_events_back_after_cut4 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test4_weight_signal[ n_events_signal_after_cut4].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM4_weight_signal[ n_events_SM_after_cut4 ].sum() / test_size )
print('Recall Score',recall_score(y_test4,y_pred_4, average = 'weighted') )
print('Precision Score',precision_score(y_test4,y_pred_4, average = 'weighted') )

ax[0,0].text(best_cut( y_test4, predict_proba_TraningAnomalo1_overAnomalo4 )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,0].set_ylabel( "Events (Log Scale)" )
ax[0,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,0].legend( loc = "upper right", fontsize = 16 )
ax[0,0].set_yscale( 'log' )
ax[0,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,0] )


predict_dados = bst1_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,1].hist( predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test1_weight_backgr/test_size )
ax[0,1].hist( predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=20 \times 10^{-6}$', weights = DataSet_Test1_weight_signal/test_size, color = 'green' )
ax[0,1].hist( predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM1_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,1].plot( [ best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight),best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut1 = predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 0 ][:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut1 = predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 1 ][:,1] > best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight )
n_events_SM_after_cut1 = predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 2 ][:,1] > best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut1 = predict_dados[ predict_dados >= best_cut( y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ]

print(' --------------- Anomalo 1 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test1_weight_backgr[ n_events_back_after_cut1 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test1_weight_signal[ n_events_signal_after_cut1].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM1_weight_signal[ n_events_SM_after_cut1 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut1 ) )
print('Recall Score',recall_score(y_test1,y_pred_1, average = 'weighted') )
print('Precision Score',precision_score(y_test1,y_pred_1, average = 'weighted') )

ax[0,1].text(best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,1].legend( loc = "upper right", fontsize = 16 )
ax[0,1].set_yscale( 'log' )
ax[0,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,1] )

predict_dados = bst1_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,0].hist( predict_proba_TraningAnomalo1_overAnomalo3[ y_test3 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'gold', label = 'Background', weights = DataSet_Test3_weight_backgr/test_size )
ax[1,0].hist( predict_proba_TraningAnomalo1_overAnomalo3[ y_test3 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', weights = DataSet_Test3_weight_signal/test_size, color = 'green' )
ax[1,0].hist( predict_proba_TraningAnomalo1_overAnomalo3[ y_test3 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM3_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,0].plot( [ best_cut(y_test3,predict_proba_TraningAnomalo1_overAnomalo3),best_cut(y_test3,predict_proba_TraningAnomalo1_overAnomalo3)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test3, predict_proba_TraningAnomalo1_overAnomalo3) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test3,predict_proba_TraningAnomalo1_overAnomalo3 ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut3 = predict_proba_TraningAnomalo1_overAnomalo3[ y_test3 == 0 ][:,1] >= best_cut( y_test3, predict_proba_TraningAnomalo1_overAnomalo3 ) 
n_events_signal_after_cut3 = predict_proba_TraningAnomalo1_overAnomalo3[ y_test3 == 1 ][:,1] >= best_cut(y_test3,predict_proba_TraningAnomalo1_overAnomalo3 )
n_events_SM_after_cut3 = predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 2 ][:,1] >= best_cut(y_test3,predict_proba_TraningAnomalo1_overAnomalo3 )
n_eventos_Data_after_cut3 = predict_dados[ predict_dados >= best_cut( y_test3,predict_proba_TraningAnomalo1_overAnomalo3 ) ]

print(' --------------- Anomalo 3 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test3_weight_backgr[ n_events_back_after_cut3 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test3_weight_signal[ n_events_signal_after_cut3].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM3_weight_signal[ n_events_SM_after_cut3 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut3 ) )
print('Recall Score',recall_score(y_test3,y_pred_3, average = 'weighted') )
print('Precision Score',precision_score(y_test3,y_pred_3, average = 'weighted') )

ax[1,0].text(best_cut( y_test3, predict_proba_TraningAnomalo1_overAnomalo3 )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,0].legend( loc = "upper right", fontsize = 16 )
ax[1,0].set_yscale( 'log' )
ax[1,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,0] )

predict_dados = bst1_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,1].hist( predict_proba_TraningAnomalo1_overAnomalo2[ y_test2 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'gold', label = 'Background', weights = DataSet_Test2_weight_backgr/test_size )
ax[1,1].hist( predict_proba_TraningAnomalo1_overAnomalo2[ y_test2 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', weights = DataSet_Test2_weight_signal/test_size, color = 'green' )
ax[1,1].hist( predict_proba_TraningAnomalo1_overAnomalo2[ y_test2 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standar Model', weights = DataSet_TestSM2_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,1].plot( [ best_cut(y_test2,predict_proba_TraningAnomalo1_overAnomalo2),best_cut(y_test2,predict_proba_TraningAnomalo1_overAnomalo2)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test2, predict_proba_TraningAnomalo1_overAnomalo2) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test2,predict_proba_TraningAnomalo1_overAnomalo2 ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut2 = predict_proba_TraningAnomalo1_overAnomalo2[ y_test2 == 0 ][:,1] > best_cut( y_test2, predict_proba_TraningAnomalo1_overAnomalo2 ) 
n_events_signal_after_cut2 = predict_proba_TraningAnomalo1_overAnomalo2[ y_test2 == 1 ][:,1] > best_cut(y_test2,predict_proba_TraningAnomalo1_overAnomalo2 )
n_events_SM_after_cut2 = predict_proba_TraningAnomalo1_overAnomalo2[ y_test2 == 2 ][:,1] > best_cut(y_test2,predict_proba_TraningAnomalo1_overAnomalo2 )
n_eventos_Data_after_cut2 = predict_dados[ predict_dados > best_cut( y_test2,predict_proba_TraningAnomalo1_overAnomalo2 ) ]

print(' --------------- Anomalo 2 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test2_weight_backgr[ n_events_back_after_cut2 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test2_weight_signal[ n_events_signal_after_cut2 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM2_weight_signal[ n_events_SM_after_cut2 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut2 ) )
print('Recall Score',recall_score(y_test2,y_pred_2, average = 'weighted') )
print('Precision Score',precision_score(y_test2,y_pred_2, average = 'weighted') )

ax[1,1].set_xlabel( "Probability" )
ax[1,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,1].legend( loc = "upper right", fontsize = 16 )
ax[1,1].set_yscale( 'log' )
ax[1,1].text(best_cut( y_test2, predict_proba_TraningAnomalo1_overAnomalo2 )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,1] )

In [ ]:
sys.exit()

In [ ]:
SM['class_predict8'] = ( bst8_binary_logloss_scale_pos_weight.predict( SM[select_columns] )[:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ).astype(int)
SM['class_predict7'] = ( bst7_binary_logloss_scale_pos_weight.predict( SM[select_columns] )[:,1] > best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ).astype(int)
SM['class_predict6'] = ( bst6_binary_logloss_scale_pos_weight.predict( SM[select_columns] )[:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ).astype(int)
SM['class_predict5'] = ( bst5_binary_logloss_scale_pos_weight.predict( SM[select_columns] )[:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ).astype(int)

SM['class_predict1'] = ( bst1_binary_logloss_scale_pos_weight.predict( SM[select_columns] )[:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ).astype(int)
SM['class_predict4'] = ( bst4_binary_logloss_scale_pos_weight.predict( SM[select_columns] )[:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ).astype(int)
SM['class_predict3'] = ( bst3_binary_logloss_scale_pos_weight.predict( SM[select_columns] )[:,1] > best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ).astype(int)
SM['class_predict2'] = ( bst2_binary_logloss_scale_pos_weight.predict( SM[select_columns] )[:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ).astype(int)

In [ ]:
ANOMALO8['class_predict'] = ( bst8_binary_logloss_scale_pos_weight.predict( ANOMALO8[select_columns] )[:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ).astype(int)
ANOMALO7['class_predict'] = ( bst7_binary_logloss_scale_pos_weight.predict( ANOMALO7[select_columns] )[:,1] > best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ).astype(int)
ANOMALO1['class_predict'] = ( bst1_binary_logloss_scale_pos_weight.predict( ANOMALO1[select_columns] )[:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ).astype(int)
ANOMALO4['class_predict'] = ( bst4_binary_logloss_scale_pos_weight.predict( ANOMALO4[select_columns] )[:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ).astype(int)
ANOMALO6['class_predict'] = ( bst6_binary_logloss_scale_pos_weight.predict( ANOMALO6[select_columns] )[:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ).astype(int)
ANOMALO5['class_predict'] = ( bst5_binary_logloss_scale_pos_weight.predict( ANOMALO5[select_columns] )[:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ).astype(int)
ANOMALO3['class_predict'] = ( bst3_binary_logloss_scale_pos_weight.predict( ANOMALO3[select_columns] )[:,1] > best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ).astype(int)
ANOMALO2['class_predict'] = ( bst2_binary_logloss_scale_pos_weight.predict( ANOMALO2[select_columns] )[:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ).astype(int)

In [ ]:
# Fazendo um dicionário de tal forma que pudesse alocar todos os anomalos numa mesma estrutura apenas para aqueles classificados como sinal (signal region)
labels_signals = [r'ANOMALO8', r'ANOMALO7', r'ANOMALO1',r'ANOMALO4']
df_signals_protons_multiRP_events_sigreg = {}
df_signals_protons_multiRP_events_sigreg[labels_signals[0]] = ANOMALO8[ ANOMALO8['class_predict'] == 1 ]
df_signals_protons_multiRP_events_sigreg[labels_signals[1]] = ANOMALO7[ ANOMALO7['class_predict'] == 1 ]
df_signals_protons_multiRP_events_sigreg[labels_signals[2]] = ANOMALO1[ ANOMALO1['class_predict'] == 1 ]
df_signals_protons_multiRP_events_sigreg[labels_signals[3]] = ANOMALO4[ ANOMALO4['class_predict'] == 1 ]

In [ ]:
DataSet_multiRP_DrellYan['class_predict8'] = ( bst8_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_DrellYan[select_columns] )[:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_QCD['class_predict8'] = ( bst8_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_QCD[select_columns] )[:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_single_top['class_predict8'] = ( bst8_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_single_top[select_columns] )[:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_TTbar['class_predict8'] = ( bst8_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_TTbar[select_columns] )[:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_VV_inclusivo['class_predict8'] = ( bst8_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_VV_inclusivo[select_columns] )[:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_WJets['class_predict8'] = ( bst8_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_WJets[select_columns] )[:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_back_multirp['class_predict8'] = ( bst8_binary_logloss_scale_pos_weight.predict( data_set_back_multirp[select_columns] )[:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ).astype(int)

DataSet_multiRP_DrellYan['class_predict7'] = ( bst7_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_DrellYan[select_columns])[:,1] > best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_QCD['class_predict7'] = ( bst7_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_QCD[select_columns] )[:,1] > best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_single_top['class_predict7'] = ( bst7_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_single_top[select_columns] )[:,1] > best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_TTbar['class_predict7'] = ( bst7_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_TTbar[select_columns] )[:,1] > best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_VV_inclusivo['class_predict7'] = ( bst7_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_VV_inclusivo[select_columns] )[:,1] > best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_WJets['class_predict7'] = ( bst7_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_WJets[select_columns] )[:,1] > best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_back_multirp['class_predict7'] = ( bst7_binary_logloss_scale_pos_weight.predict( data_set_back_multirp[select_columns] )[:,1] > best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ).astype(int)

DataSet_multiRP_DrellYan['class_predict6'] = ( bst6_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_DrellYan[select_columns])[:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_QCD['class_predict6'] = ( bst6_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_QCD[select_columns] )[:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_single_top['class_predict6'] = ( bst6_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_single_top[select_columns] )[:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_TTbar['class_predict6'] = ( bst6_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_TTbar[select_columns] )[:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_VV_inclusivo['class_predict6'] = ( bst6_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_VV_inclusivo[select_columns] )[:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_WJets['class_predict6'] = ( bst6_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_WJets[select_columns] )[:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_back_multirp['class_predict6'] = ( bst6_binary_logloss_scale_pos_weight.predict( data_set_back_multirp[select_columns] )[:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ).astype(int)

DataSet_multiRP_DrellYan['class_predict5'] = ( bst5_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_DrellYan[select_columns])[:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_QCD['class_predict5'] = ( bst5_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_QCD[select_columns] )[:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_single_top['class_predict5'] = ( bst5_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_single_top[select_columns] )[:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_TTbar['class_predict5'] = ( bst5_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_TTbar[select_columns] )[:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_VV_inclusivo['class_predict5'] = ( bst5_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_VV_inclusivo[select_columns] )[:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_WJets['class_predict5'] = ( bst5_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_WJets[select_columns] )[:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_back_multirp['class_predict5'] = ( bst5_binary_logloss_scale_pos_weight.predict( data_set_back_multirp[select_columns] )[:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ).astype(int)

DataSet_multiRP_DrellYan['class_predict1'] = ( bst1_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_DrellYan[select_columns] )[:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_QCD['class_predict1'] = ( bst1_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_QCD[select_columns] )[:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_single_top['class_predict1'] = ( bst1_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_single_top[select_columns] )[:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_TTbar['class_predict1'] = ( bst1_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_TTbar[select_columns] )[:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_VV_inclusivo['class_predict1'] = ( bst1_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_VV_inclusivo[select_columns] )[:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_WJets['class_predict1'] = ( bst1_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_WJets[select_columns] )[:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_back_multirp['class_predict1'] = ( bst1_binary_logloss_scale_pos_weight.predict( data_set_back_multirp[select_columns]  )[:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ).astype(int)

DataSet_multiRP_DrellYan['class_predict2'] = ( bst2_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_DrellYan[select_columns] )[:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_QCD['class_predict2'] = ( bst2_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_QCD[select_columns] )[:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_single_top['class_predict2'] = ( bst2_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_single_top[select_columns] )[:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_TTbar['class_predict2'] = ( bst2_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_TTbar[select_columns] )[:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_VV_inclusivo['class_predict2'] = ( bst2_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_VV_inclusivo[select_columns] )[:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_WJets['class_predict2'] = ( bst2_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_WJets[select_columns] )[:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_back_multirp['class_predict2'] = ( bst2_binary_logloss_scale_pos_weight.predict( data_set_back_multirp[select_columns] )[:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ).astype(int)

DataSet_multiRP_DrellYan['class_predict3'] = ( bst3_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_DrellYan[select_columns] )[:,1] > best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_QCD['class_predict3'] = ( bst3_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_QCD[select_columns] )[:,1] > best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_single_top['class_predict3'] = ( bst3_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_single_top[select_columns] )[:,1] > best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_TTbar['class_predict3'] = ( bst3_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_TTbar[select_columns] )[:,1] > best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_VV_inclusivo['class_predict3'] = ( bst3_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_VV_inclusivo[select_columns] )[:,1] > best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_WJets['class_predict3'] = ( bst3_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_WJets[select_columns] )[:,1] > best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_back_multirp['class_predict3'] = ( bst3_binary_logloss_scale_pos_weight.predict( data_set_back_multirp[select_columns]  )[:,1] > best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ).astype(int)

DataSet_multiRP_DrellYan['class_predict4'] = ( bst4_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_DrellYan[select_columns] )[:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_QCD['class_predict4'] = ( bst4_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_QCD[select_columns] )[:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_single_top['class_predict4'] = ( bst4_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_single_top[select_columns] )[:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_TTbar['class_predict4'] = ( bst4_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_TTbar[select_columns] )[:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_VV_inclusivo['class_predict4'] = ( bst4_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_VV_inclusivo[select_columns] )[:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ).astype(int)
DataSet_multiRP_WJets['class_predict4'] = ( bst4_binary_logloss_scale_pos_weight.predict( DataSet_multiRP_WJets[select_columns] )[:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_back_multirp['class_predict4'] = ( bst4_binary_logloss_scale_pos_weight.predict( data_set_back_multirp[select_columns] )[:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ).astype(int)

In [ ]:
data_set_dados_multirp['class_predict8'] = ( bst8_binary_logloss_scale_pos_weight.predict( data_set_dados_multirp[select_columns] )[:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_dados_multirp['class_predict7'] = ( bst7_binary_logloss_scale_pos_weight.predict( data_set_dados_multirp[select_columns] )[:,1] > best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_dados_multirp['class_predict1'] = ( bst1_binary_logloss_scale_pos_weight.predict( data_set_dados_multirp[select_columns] )[:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_dados_multirp['class_predict4'] = ( bst4_binary_logloss_scale_pos_weight.predict( data_set_dados_multirp[select_columns] )[:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_dados_multirp['class_predict5'] = ( bst5_binary_logloss_scale_pos_weight.predict( data_set_dados_multirp[select_columns] )[:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_dados_multirp['class_predict2'] = ( bst2_binary_logloss_scale_pos_weight.predict( data_set_dados_multirp[select_columns] )[:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_dados_multirp['class_predict6'] = ( bst6_binary_logloss_scale_pos_weight.predict( data_set_dados_multirp[select_columns] )[:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ).astype(int)
data_set_dados_multirp['class_predict3'] = ( bst3_binary_logloss_scale_pos_weight.predict( data_set_dados_multirp[select_columns] )[:,1] > best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ).astype(int)

# Método ABCD

# Estiamando o Background

A partir das duas variáveis $ \left (  \dfrac{M_{WW}}{M_{PPS}} - 1  \;\; \text{e} \;\; Y_{WW} - Y_{PPS} \right ) $ que têm distribuições não correlacionadas para o background. Um corte em cada uma dessas variáveis terá uma eficiência independente de quaisquer seleções feitas na outra variável. Digamos que dividamos o plano 2D em quatro regiões (A, B, C, D) sendo A a região do sinal final onde os requisitos de seleção das variáveis são aplicados.

Se as seleções são de fato não correlacionadas para o background, então pode-se supor que a proporção de eventos em cada região pode ser relacionada como: 

$$
\dfrac{ N_{A}^{bkg} }{ N_{B}^{bkg} } = \dfrac{ N_{C}^{bkg} }{ N_{D}^{bkg} } \Leftrightarrow N_{A}^{bkg} = N_{B}^{bkg} \cdot \dfrac{N_{C}^{bkg}}{N_{D}^{bkg}}
$$

## Para as amostras de Background

In [ ]:
df_protons_multiRP_bkg_events = data_set_back_multirp
df_protons_multiRP_bkg_events['shiftedRatioMWW_MX'] =  df_protons_multiRP_bkg_events[ "Mww/Mx" ] - 1 
df_protons_multiRP_bkg_events['diffYWW_YX'] = df_protons_multiRP_bkg_events['Yww_Yx'] 

cutMww_Mx = 0.30
cutYww_Yx = 0.50 

msk_bkg_cut1 = ( np.abs( df_protons_multiRP_bkg_events[ "shiftedRatioMWW_MX" ] ) <= cutMww_Mx )
msk_bkg_cut2 = ( np.abs( df_protons_multiRP_bkg_events[ "diffYWW_YX" ] ) <= cutYww_Yx )
msk_bkg_A =  msk_bkg_cut1 &  msk_bkg_cut2
msk_bkg_B = ~msk_bkg_cut1 &  msk_bkg_cut2
msk_bkg_C =  msk_bkg_cut1 & ~msk_bkg_cut2
msk_bkg_D = ~msk_bkg_cut1 & ~msk_bkg_cut2

n_events_bkg   = np.sum( df_protons_multiRP_bkg_events[ "weight" ]  )
n_events_bkg_A = np.sum( df_protons_multiRP_bkg_events[ msk_bkg_A ][ "weight" ] )
n_events_bkg_B = np.sum( df_protons_multiRP_bkg_events[ msk_bkg_B ][ "weight" ])
n_events_bkg_C = np.sum( df_protons_multiRP_bkg_events[ msk_bkg_C ][ "weight" ])
n_events_bkg_D = np.sum( df_protons_multiRP_bkg_events[ msk_bkg_D ][ "weight" ])

print ( "Number of events for background: {}".format( n_events_bkg ) )
print ( "Number of events (A): {}".format( n_events_bkg_A ) )
print ( "Number of events (B): {}".format( n_events_bkg_B ) )
print ( "Number of events (C): {}".format( n_events_bkg_C ) )
print ( "Number of events (D): {}".format( n_events_bkg_D ), '\n')

x_min = -1.2
x_max = 3.
y_min = -3.5
y_max =  3.5
x_center = ( x_min + x_max ) / 2.
y_center = ( y_min + y_max ) / 2.

fig = plt.figure( figsize=(16,10) )
plt.plot( (x_min,x_max), (y_center,y_center), color='blue' )
plt.plot( (0,0), (y_min,y_max), color='blue' )
ax = plt.gca()
label_str_ = r"A ($|M_{WW} / M_{PPS}| < 0.30$  $|Y_{WW} - Y_{PPS}| < 0.50$)"
df_protons_multiRP_bkg_events[ msk_bkg_A ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lime', label=label_str_ )
label_str_ = r"B ($|M_{WW} / M_{PPS}| > 0.30$  $|Y_{WW} - Y_{PPS}| < 0.50$)"
df_protons_multiRP_bkg_events[ msk_bkg_B ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='green', label=label_str_ )
label_str_ = r"C ($|M_{WW} / M_{PPS}| < 0.30$  $|Y_{WW} - Y_{PPS}| > 0.50$)"
df_protons_multiRP_bkg_events[ msk_bkg_C ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='green', label=label_str_ )
label_str_ = r"D ($|M_{WW} / M_{PPS}| > 0.30$  $|Y_{WW} - Y_{PPS}| > 0.50$)"
df_protons_multiRP_bkg_events[ msk_bkg_D ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lightcoral', label=label_str_ )
plt.legend( loc='best', fontsize=12, framealpha=0.9, frameon=True, fancybox=True )
ax.text( 0.80, 0.95, "Background", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes, fontsize=18 )
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xlabel( r"$\dfrac{M_{WW}}{M_{PPS}} - 1$", fontsize=22 )
plt.ylabel( r"$Y_{WW} - Y_{PPS}$", fontsize=22 )
hep.cms.label(llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$" )
#plt.savefig( pwd_savefig+'ABCD_multiRP_bkg_events.pdf' )
#plt.show()
#plt.close()

## Para os Dados

In [ ]:
df_protons_multiRP_data_events = data_set_dados_multirp
df_protons_multiRP_data_events['shiftedRatioMWW_MX'] =  df_protons_multiRP_data_events[ "Mww/Mx" ] - 1 
df_protons_multiRP_data_events['diffYWW_YX'] = df_protons_multiRP_data_events['Yww_Yx'] 


msk_data_cut1 = ( np.abs( df_protons_multiRP_data_events[ "shiftedRatioMWW_MX" ] ) <= 0.30 )
msk_data_cut2 = ( np.abs( df_protons_multiRP_data_events[ "diffYWW_YX" ] ) <= 0.50 )
msk_data_A =  msk_data_cut1 &  msk_data_cut2
msk_data_B = ~msk_data_cut1 &  msk_data_cut2
msk_data_C =  msk_data_cut1 & ~msk_data_cut2
msk_data_D = ~msk_data_cut1 & ~msk_data_cut2
msk_data = ~msk_data_A


n_events_data_B = df_protons_multiRP_data_events[ msk_data_B ].shape[0]
n_events_data_C = df_protons_multiRP_data_events[ msk_data_C ].shape[0]
n_events_data_D = df_protons_multiRP_data_events[ msk_data_D ].shape[0]
resample_factor = 20
print ( "Number of events for Data (B): {} - Ratio: {}".format( n_events_data_B, ( n_events_data_B / ( n_events_bkg_B / resample_factor ) ) ) )
print ( "Number of events for Data (C): {} - Ratio: {}".format( n_events_data_C, ( n_events_data_C / ( n_events_bkg_C / resample_factor ) ) ) )
print ( "Number of events for Data (D): {} - Ratio: {}".format( n_events_data_D, ( n_events_data_D / ( n_events_bkg_D / resample_factor ) ) ), '\n' )


x_min = -1.5
x_max =  1.5
y_min = -3.0
y_max =  3.0
x_center = ( x_min + x_max ) / 2.
y_center = ( y_min + y_max ) / 2.

fig = plt.figure( figsize=(15,10) )
plt.plot( (x_min,x_max), (y_center,y_center), color='blue' )
plt.plot( (x_center,x_center), (y_min,y_max), color='blue' )
ax = plt.gca()
df_protons_multiRP_data_events[ msk_data_B ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='green', label = label_str_ )
label_str_ = r"B ($|M_{WW} / M_{PPS} - 1| > 0.30$  $|Y_{WW} - Y_{PPS}| < 0.50$)"
df_protons_multiRP_data_events[ msk_data_C ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='green', label = label_str_ )
label_str_ = r"C ($|M_{WW} / M_{PPS} - 1| < 0.30$  $|Y_{WW} - Y_{PPS}| > 0.50$)"
df_protons_multiRP_data_events[ msk_data_D ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lightcoral', label = label_str_ )
label_str_ = r"D ($|M_{WW} / M_{PPS} - 1| > 0.30$  $|Y_{WW} - Y_{PPS}| > 0.50$"
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xlabel( r"$\dfrac{M_{WW}}{ M_{PPS} } - 1$", fontsize=22 )
plt.ylabel( r"$Y_{WW} - Y_{PPS}$", fontsize=22 )
plt.legend( loc='best', fontsize=12, framealpha=0.9, frameon=True, fancybox=True )
ax.text( -1.3, 2, "Data-2016", fontsize=18 )
hep.cms.label(llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$" )
#plt.savefig( pwd_savefig+'ABCD_multiRP_data_events.pdf' )
#plt.show()
#plt.close()

## Para os eventos de background na região de Background após o resultado da classificação do LightGBM com o anômalo 8

In [ ]:
df_protons_multiRP_bkg_events_bkgreg = df_protons_multiRP_bkg_events[ df_protons_multiRP_bkg_events[ 'class_predict8' ] == 0 ]

msk_bkg_cut1 = ( np.abs( df_protons_multiRP_bkg_events_bkgreg[ "shiftedRatioMWW_MX" ] ) <= 0.30 )
msk_bkg_cut2 = ( np.abs( df_protons_multiRP_bkg_events_bkgreg[ "diffYWW_YX" ] ) <= 0.50 )
msk_bkg_A =  msk_bkg_cut1 &  msk_bkg_cut2
msk_bkg_B = ~msk_bkg_cut1 &  msk_bkg_cut2
msk_bkg_C =  msk_bkg_cut1 & ~msk_bkg_cut2
msk_bkg_D = ~msk_bkg_cut1 & ~msk_bkg_cut2

n_events_bkg_bkgreg   = np.sum(df_protons_multiRP_bkg_events_bkgreg[ "weight" ])
n_events_bkg_bkgreg_A = np.sum(df_protons_multiRP_bkg_events_bkgreg[ msk_bkg_A ][ "weight" ] )
n_events_bkg_bkgreg_B = np.sum(df_protons_multiRP_bkg_events_bkgreg[ msk_bkg_B ][ "weight" ]  )
n_events_bkg_bkgreg_C = np.sum(df_protons_multiRP_bkg_events_bkgreg[ msk_bkg_C ][ "weight" ] )
n_events_bkg_bkgreg_D = np.sum(df_protons_multiRP_bkg_events_bkgreg[ msk_bkg_D ][ "weight" ])

print ( "Number of events of the background on Signal Region: {}".format( n_events_bkg_bkgreg ) )
print ( "Number of events (A): {}".format( n_events_bkg_bkgreg_A ) )
print ( "Number of events (B): {}".format( n_events_bkg_bkgreg_B ) )
print ( "Number of events (C): {}".format( n_events_bkg_bkgreg_C ) )
print ( "Number of events (D): {}".format( n_events_bkg_bkgreg_D ),'\n' )

x_min = -1.5
x_max =  1.5
y_min = -3.5
y_max =  3.5
x_center = ( x_min + x_max ) / 2.
y_center = ( y_min + y_max ) / 2.

fig = plt.figure( figsize=(18,10) )
plt.plot( (x_min,x_max), (y_center,y_center), color='blue' )
plt.plot( (x_center,x_center), (y_min,y_max), color='blue' )
ax = plt.gca()
label_str_ = r"A ($|M_{WW} / M_{PPS} - 1| < 0.30$  $|Y_{WW} - Y_{PPS}| < 0.50$)"
df_protons_multiRP_bkg_events_bkgreg[ msk_bkg_A ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lime', label=label_str_ )
label_str_ = r"B ($|M_{WW} / M_{PPS} - 1| > 0.30$  $|Y_{WW} - Y_{PPS}| < 0.50$)"
df_protons_multiRP_bkg_events_bkgreg[ msk_bkg_B ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='green', label=label_str_ )
label_str_ = r"C ($|M_{WW} / M_{PPS} - 1| < 0.30$  $|Y_{WW} - Y_{PPS}| > 0.50$)"
df_protons_multiRP_bkg_events_bkgreg[ msk_bkg_C ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='green', label=label_str_ )
label_str_ = r"D ($|M_{WW} / M_{PPS} - 1| > 0.30$  $|Y_{WW} - Y_{PPS}| > 0.50$)"
df_protons_multiRP_bkg_events_bkgreg[ msk_bkg_D ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lightcoral', label=label_str_ )
plt.legend( loc='best', fontsize=12, framealpha=0.9, frameon=True, fancybox=True )
title_str_ = "Background \n$\\rm{{Cut Prob.}} < {:.2f}$\n\nBackground Region".format( best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight) )
title_str_ = r"{}".format( title_str_ )
ax.text( -1.4, 2, title_str_, fontsize=18 )
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
#plt.title(r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $')
plt.xlabel( r"$\dfrac{M_{WW}}{M_{PPS}} - 1$", fontsize=22 )
plt.ylabel( r"$Y_{WW} - Y_{PPS}$", fontsize=22 )
#plt.savefig( pwd_savefig+'ABCD_multiRP_bkg_events_BACKGROUND_REGION.pdf' )
#plt.show()
#plt.close()

## Para o Background na Região de Signal 

In [ ]:
labels_signals = [r'ANOMALO8', r'ANOMALO7', r'ANOMALO1',r'ANOMALO4']
df_protons_multiRP_bkg_events_sigreg = {}
df_protons_multiRP_bkg_events_sigreg[labels_signals[0]] = data_set_back_multirp[ data_set_back_multirp['class_predict8'] == 1 ]
df_protons_multiRP_bkg_events_sigreg[labels_signals[1]] = data_set_back_multirp[ data_set_back_multirp['class_predict7'] == 1 ]
df_protons_multiRP_bkg_events_sigreg[labels_signals[2]] = data_set_back_multirp[ data_set_back_multirp['class_predict1'] == 1 ]
df_protons_multiRP_bkg_events_sigreg[labels_signals[3]] = data_set_back_multirp[ data_set_back_multirp['class_predict4'] == 1 ]

labels_samples = [
    r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $',
    r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $',
    r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=2.0 \times 10^{-5}$',
    r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=8 \times 10^{-6}$' ]

nrows_ = ( int( len(labels_signals)/2 ) + 1 if ( len(labels_signals) % 2 ) > 0 else  int( len(labels_signals)/2 ) )
fig, axes = plt.subplots( nrows_, 2, figsize=(24,nrows_*10) )

row_ = 0
col_ = 0
for label_ in labels_signals:
    print ( label_ )
    
    df_protons_multiRP_bkg_events_sigreg[ label_ ]['shiftedRatioMWW_MX'] = df_protons_multiRP_bkg_events_sigreg[ label_ ]['Mww/Mx'] - 1
    msk_sig_cut1 = ( np.abs( df_protons_multiRP_bkg_events_sigreg[ label_ ][ "shiftedRatioMWW_MX" ] ) <= 0.30 )
    msk_sig_cut2 = ( np.abs( df_protons_multiRP_bkg_events_sigreg[ label_ ][ "Yww_Yx" ] ) <= 0.50 )
    msk_sig_A =  msk_sig_cut1 &  msk_sig_cut2
    msk_sig_B = ~msk_sig_cut1 &  msk_sig_cut2
    msk_sig_C =  msk_sig_cut1 & ~msk_sig_cut2
    msk_sig_D = ~msk_sig_cut1 & ~msk_sig_cut2

    n_events_sig   = np.sum( df_protons_multiRP_bkg_events_sigreg[ label_ ][ "weight" ] )
    err_n_events_sig = np.sqrt( np.sum(np.square(  df_protons_multiRP_bkg_events_sigreg[ label_ ][ "weight" ] ) ) )
    n_events_sig_A = np.sum( df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_A ][ "weight" ] )
    err_n_events_sig_A = np.sqrt( np.sum(np.square( df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_A ][ "weight" ]  ) ) )
    n_events_sig_B = np.sum( df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_B ][ "weight" ] )
    err_n_events_sig_B = np.sqrt( np.sum(np.square(  df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_B ][ "weight" ] ) ) )
    n_events_sig_C = np.sum(  df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_C ][ "weight" ] )
    err_n_events_sig_C = np.sqrt( np.sum(np.square( df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_C ][ "weight" ] ) ) )
    n_events_sig_D = np.sum( df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_D ][ "weight" ] )
    err_n_events_sig_D = np.sqrt( np.sum(np.square( df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_D ][ "weight" ] ) ) )

    print ( "Number of events: {} +/- {}".format( n_events_sig, err_n_events_sig ) )
    print ( "Number of events (A): {} +/- {}".format( n_events_sig_A, err_n_events_sig_A ) )
    print ( "Number of events (B): {} +/- {}".format( n_events_sig_B, err_n_events_sig_B ) )
    print ( "Number of events (C): {} +/- {}".format( n_events_sig_C, err_n_events_sig_C ) )
    print ( "Number of events (D): {} +/- {}".format( n_events_sig_D, err_n_events_sig_D ) )

    x_min = -1.0
    x_max =  1.0
    y_min = -3.0
    y_max =  3.0
    x_center = ( x_min + x_max ) / 2.
    y_center = ( y_min + y_max ) / 2.

    print ( row_, col_ )
    axes[ row_, col_ ].plot( (x_min,x_max), (y_center,y_center), color='blue' )
    axes[ row_, col_ ].plot( (x_center,x_center), (y_min,y_max), color='blue' )
    if np.sum( msk_sig_A ) > 0:
        label_str_ = r"A ($|M_{WW} / M_{pp} - 1| < 0.30$  $|Y_{WW} - Y_{pp}| < 0.50$)"
        df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_A ].plot( "shiftedRatioMWW_MX", "Yww_Yx", 'scatter', ax=axes[ row_, col_ ], color='lime', label = label_str_ )
    if np.sum( msk_sig_B ) > 0:
        label_str_ = r"B ($|M_{WW} / M_{pp} - 1| > 0.30$  $|Y_{WW} - Y_{pp}| < 0.50$)"
        df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_B ].plot( "shiftedRatioMWW_MX", "Yww_Yx", 'scatter', ax=axes[ row_, col_ ], color='green', label = label_str_ )
    if np.sum( msk_sig_C ) > 0:
        label_str_ = r"C ($|M_{WW} / M_{pp} - 1| < 0.30$  $|Y_{WW} - Y_{pp}| > 0.50$)"
        df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_C ].plot( "shiftedRatioMWW_MX", "Yww_Yx", 'scatter', ax=axes[ row_, col_ ], color='green', label = label_str_ )
    if np.sum( msk_sig_D ) > 0:
        label_str_ = r"D ($|M_{WW} / M_{pp} - 1| > 0.30$  $|Y_{WW} - Y_{pp}| > 0.50$)"
        df_protons_multiRP_bkg_events_sigreg[ label_ ][ msk_sig_D ].plot( "shiftedRatioMWW_MX", "Yww_Yx", 'scatter', ax=axes[ row_, col_ ], color='lightcoral', label = label_str_ )
    #title_str_ = "{}\n$\\rm{{Prob.}} > {:.2f}$".format( labels_samples[ label_ ], prob_cut )
    #title_str_ = r"{}".format( title_str_ )
    #axes[ row_, col_ ].text( 0.70, 0.92, title_str_, horizontalalignment='left', verticalalignment='center', transform=axes[ row_, col_ ].transAxes, fontsize=18 )
    axes[ row_, col_ ].set_xlim(x_min, x_max)
    axes[ row_, col_ ].set_ylim(y_min, y_max)
    axes[ row_, col_ ].set_xlabel( r"$\dfrac{M_{WW}}{M_{PPS}} - 1$", fontsize=22 )
    axes[ row_, col_ ].set_ylabel( r"$Y_{WW} - Y_{PPS}$", fontsize=22 )
    axes[ row_, col_ ].set_title( "Background Sample", fontsize=22 )
    axes[ 0 , 0].text( -0.9, 2.4 , labels_samples[0], fontsize=18)
    axes[ 0 , 0].text( 0.5, 2.3 , 'Signal Region\nCut Prob > {:2.3f}'.format(best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight)), fontsize=18) 
    axes[ 0 , 1].text( -0.9, 2.4 , labels_samples[1], fontsize=18)
    axes[ 0 , 1].text( 0.5, 2.3 , 'Signal Region\nCut Prob > {:2.3f}'.format(best_cut(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight)), fontsize=18)
    axes[ 1 , 0].text( -0.9, 2.4 , labels_samples[2], fontsize=18)
    axes[ 1 , 0].text( 0.5, 2.3 , 'Signal Region\nCut Prob > {:2.3f}'.format(best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight)), fontsize=18)
    axes[ 1 , 1].text( -0.9, 2.4 , labels_samples[3], fontsize=18)
    axes[ 1 , 1].text( 0.5, 2.3 , 'Signal Region\nCut Prob > {:2.3f}'.format(best_cut(y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight)), fontsize=18)
    axes[ 0 , 0].legend(loc='lower right', fontsize=13,framealpha=0.9, frameon=True, fancybox=True)
    axes[ 0 , 1].legend(loc='lower right', fontsize=13,framealpha=0.9, frameon=True, fancybox=True)
    axes[ 1 , 0].legend(loc='lower right', fontsize=13,framealpha=0.9, frameon=True, fancybox=True)
    axes[ 1 , 1].legend(loc='lower right', fontsize=13,framealpha=0.9, frameon=True, fancybox=True)
    #plt.savefig(pwd_savefig+'multiRP_events_sigreg_BACKGROUND_SignalRegion.pdf')
    col_ += 1
    if col_ >= 2:
        row_ += 1
        col_  = 0

## Para o Signal na Região de Signal

In [ ]:
labels_samples = [
    r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $',
    r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $',
    r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=2.0 \times 10^{-5}$',
    r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=8 \times 10^{-6}$' ]

nrows_ = ( int( len(labels_samples)/2 ) + 1 if ( len(labels_samples) % 2 ) > 0 else  int( len(labels_samples)/2 ) )
fig, axes = plt.subplots( nrows_, 2, figsize=(24,nrows_*10) )

row_ = 0
col_ = 0
for label_ in labels_signals:
    print ( label_ )
    
    df_signals_protons_multiRP_events_sigreg[ label_ ]['shiftedRatioMWW_MX'] = df_signals_protons_multiRP_events_sigreg[ label_ ]['Mww/Mx'] - 1
    msk_sig_cut1 = ( np.abs( df_signals_protons_multiRP_events_sigreg[ label_ ][ "shiftedRatioMWW_MX" ] ) <= 0.30 )
    msk_sig_cut2 = ( np.abs( df_signals_protons_multiRP_events_sigreg[ label_ ][ "Yww_Yx" ] ) <= 0.50 )
    msk_sig_A =  msk_sig_cut1 &  msk_sig_cut2
    msk_sig_B = ~msk_sig_cut1 &  msk_sig_cut2
    msk_sig_C =  msk_sig_cut1 & ~msk_sig_cut2
    msk_sig_D = ~msk_sig_cut1 & ~msk_sig_cut2

    n_events_sig   = np.sum(  df_signals_protons_multiRP_events_sigreg[ label_ ][ "weight" ] )
    err_n_events_sig = np.sqrt( np.sum( np.square( df_signals_protons_multiRP_events_sigreg[ label_ ][ "weight" ] ) ) )
    n_events_sig_A = np.sum(  df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_A ][ "weight" ] )
    err_n_events_sig_A = np.sqrt( np.sum( np.square( df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_A ][ "weight" ]  ) ) )
    n_events_sig_B = np.sum( df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_B ][ "weight" ] )
    err_n_events_sig_B = np.sqrt( np.sum( np.square( df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_B ][ "weight" ] ) ) )
    n_events_sig_C = np.sum( df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_C ][ "weight" ] )
    err_n_events_sig_C = np.sqrt( np.sum( np.square( df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_C ][ "weight" ] ) ) )
    n_events_sig_D = np.sum(  df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_D ][ "weight" ] )
    err_n_events_sig_D = np.sqrt( np.sum( np.square( df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_D ][ "weight" ] ) ) )

    print ( "Number of events: {} +/- {}".format( n_events_sig, err_n_events_sig ) )
    print ( "Number of events (A): {} +/- {}".format( n_events_sig_A, err_n_events_sig_A ) )
    print ( "Number of events (B): {} +/- {}".format( n_events_sig_B, err_n_events_sig_B ) )
    print ( "Number of events (C): {} +/- {}".format( n_events_sig_C, err_n_events_sig_C ) )
    print ( "Number of events (D): {} +/- {}".format( n_events_sig_D, err_n_events_sig_D ) )

    x_min = -1.0
    x_max =  1.0
    y_min = -3.0
    y_max =  3.0
    x_center = ( x_min + x_max ) / 2.
    y_center = ( y_min + y_max ) / 2.

    print ( row_, col_ )
    axes[ row_, col_ ].plot( (x_min,x_max), (y_center,y_center), color='blue' )
    axes[ row_, col_ ].plot( (x_center,x_center), (y_min,y_max), color='blue' )
    if np.sum( msk_sig_A ) > 0:
        label_str_ = r"A ($|M_{WW} / M_{PPS} - 1| < 0.30$  $|Y_{WW} - Y_{PPS}| < 0.50$)"
        df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_A ].plot( "shiftedRatioMWW_MX", "Yww_Yx", 'scatter', ax=axes[ row_, col_ ], color='lime', label = label_str_ )
    if np.sum( msk_sig_B ) > 0:
        label_str_ = r"B ($|M_{WW} / M_{PPS} - 1| > 0.30$  $|Y_{WW} - Y_{PPS}| < 0.50$)"
        df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_B ].plot( "shiftedRatioMWW_MX", "Yww_Yx", 'scatter', ax=axes[ row_, col_ ], color='green', label = label_str_ )
    if np.sum( msk_sig_C ) > 0:
        label_str_ = r"C ($|M_{WW} / M_{PPS} - 1| < 0.30$  $|Y_{WW} - Y_{PPS}| > 0.50$)"
        df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_C ].plot( "shiftedRatioMWW_MX", "Yww_Yx", 'scatter', ax=axes[ row_, col_ ], color='green', label = label_str_ )
    if np.sum( msk_sig_D ) > 0:
        label_str_ = r"D ($|M_{WW} / M_{PPS} - 1| > 0.30$  $|Y_{WW} - Y_{PPS}| > 0.50$)"
        df_signals_protons_multiRP_events_sigreg[ label_ ][ msk_sig_D ].plot( "shiftedRatioMWW_MX", "Yww_Yx", 'scatter', ax=axes[ row_, col_ ], color='lightcoral', label = label_str_ )
    #title_str_ = "{}\n$\\rm{{Prob.}} > {:.2f}$".format( labels_samples[ label_ ], prob_cut )
    #title_str_ = r"{}".format( title_str_ )
    #axes[ row_, col_ ].text( 0.70, 0.92, title_str_, horizontalalignment='left', verticalalignment='center', transform=axes[ row_, col_ ].transAxes, fontsize=18 )
    axes[ row_, col_ ].set_xlim(x_min, x_max)
    axes[ row_, col_ ].set_ylim(y_min, y_max)
    axes[ row_, col_ ].set_xlabel( r"$M_{WW} / M_{PPS} - 1$", fontsize=22 )
    axes[ row_, col_ ].set_ylabel( r"$Y_{WW} - Y_{PPS}$", fontsize=22 )
    axes[ row_, col_ ].set_title( "Signal Sample", fontsize=22 )
    axes[ 0 , 0].text( -0.9, 2.4 , labels_samples[0], fontsize=18)
    axes[ 0 , 0].text( 0.5, 2.3 , 'Signal Region\nCut Prob > {:2.3f}'.format(best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight)), fontsize=18)
    axes[ 0 , 1].text( -0.9, 2.4 , labels_samples[1], fontsize=18)
    axes[ 0 , 1].text( 0.5, 2.3 , 'Signal Region\nCut Prob > {:2.3f}'.format(best_cut(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight)), fontsize=18)
    axes[ 1 , 0].text( -0.9, 2.4 , labels_samples[2], fontsize=18)
    axes[ 1 , 0].text( 0.5, 2.3 , 'Signal Region\nCut Prob > {:2.3f}'.format(best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight)), fontsize=18)
    axes[ 1 , 1].text( -0.9, 2.4 , labels_samples[3], fontsize=18)
    axes[ 1 , 1].text( 0.5, 2.3 , 'Signal Region\nCut Prob > {:2.3f}'.format(best_cut(y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight)), fontsize=18)
    axes[ 0 , 0].legend(loc='lower right', fontsize=13, framealpha=0.9, frameon=True, fancybox=True)
    axes[ 0 , 1].legend(loc='lower right', fontsize=13, framealpha=0.9, frameon=True, fancybox=True)
    axes[ 1 , 0].legend(loc='lower right', fontsize=13, framealpha=0.9, frameon=True, fancybox=True)
    axes[ 1 , 1].legend(loc='lower right', fontsize=13, framealpha=0.9, frameon=True, fancybox=True)
    #plt.savefig(pwd_savefig+'multiRP_events_sigreg_SIGNAL_SignalRegion.pdf')
    col_ += 1
    if col_ >= 2:
        row_ += 1
        col_  = 0

## Para os dados na região de background

In [ ]:
df_protons_multiRP_data_events_bkgreg = data_set_dados_multirp[ data_set_dados_multirp['class_predict8'] == 0 ]
df_protons_multiRP_data_events_bkgreg['shiftedRatioMWW_MX'] =  df_protons_multiRP_data_events_bkgreg[ "Mww/Mx" ] - 1 
df_protons_multiRP_data_events_bkgreg['diffYWW_YX'] = df_protons_multiRP_data_events_bkgreg['Yww_Yx'] 

x_min = -1.5
x_max =  1.5
y_min = -3.0
y_max =  3.0
x_center = ( x_min + x_max ) / 2.
y_center = ( y_min + y_max ) / 2.

fig = plt.figure( figsize=(19,10) )
plt.plot( (x_min,x_max), (y_center,y_center), color='blue' )
plt.plot( (x_center,x_center), (y_min,y_max), color='blue' )
ax = plt.gca()
df_protons_multiRP_data_events_bkgreg.plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='black' )
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xlabel( r"$\dfrac{M_{WW}}{M_{PPS}} - 1$", fontsize=22 )
plt.ylabel( r"$Y_{WW} - Y_{PPS}$", fontsize=22 )
#plt.savefig( pwd_savefig+'ABCD_multiRP_data_events_BACKGROUND_REGION.pdf' )
#plt.show()
#plt.close()

## Para os Dados na região de Signal

In [ ]:
df_protons_multiRP_data_events_bkgreg = data_set_dados_multirp[ data_set_dados_multirp['class_predict8'] == 1 ]
df_protons_multiRP_data_events_bkgreg['shiftedRatioMWW_MX'] =  df_protons_multiRP_data_events_bkgreg[ "Mww/Mx" ] - 1 
df_protons_multiRP_data_events_bkgreg['diffYWW_YX'] = df_protons_multiRP_data_events_bkgreg['Yww_Yx'] 

x_min = -1.5
x_max =  1.5
y_min = -3.0
y_max =  3.0
x_center = ( x_min + x_max ) / 2.
y_center = ( y_min + y_max ) / 2.

fig = plt.figure( figsize=(19,10) )
plt.plot( (x_min,x_max), (y_center,y_center), color='blue' )
plt.plot( (x_center,x_center), (y_min,y_max), color='blue' )
ax = plt.gca()
df_protons_multiRP_data_events_bkgreg.plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='black' )
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xlabel( r"$\dfrac{M_{WW}}{M_{PPS}} - 1$", fontsize=22 )
plt.ylabel( r"$Y_{WW} - Y_{PPS}$", fontsize=22 )
#plt.savefig( pwd_savefig+'ABCD_multiRP_data_events_SIGNAL_REGION.pdf' )
#plt.show()
#plt.close()

## Contagem dos eventos depois do corte

In [ ]:
DrellYan_counting = np.sum(DataSet_multiRP_DrellYan['weight'])
QCD_counting = np.sum(DataSet_multiRP_QCD['weight'])
Single_top_counting = np.sum(DataSet_multiRP_single_top['weight'])
VV_inclusive_counting = np.sum(DataSet_multiRP_VV_inclusivo['weight'])
WJets_counting = np.sum(DataSet_multiRP_WJets['weight'])
ttbar_counting = np.sum(DataSet_multiRP_TTbar['weight'])
ANOMALO8_counting = np.sum(ANOMALO8['weight'])
SM_counting = np.sum(SM['weight'])


In [ ]:
DrellYan_counting_8 = np.sum(DataSet_multiRP_DrellYan[DataSet_multiRP_DrellYan['class_predict8']==1]['weight'])
QCD_counting_8 = np.sum(DataSet_multiRP_QCD[DataSet_multiRP_QCD['class_predict8']==1]['weight'])
Single_top_counting_8 = np.sum(DataSet_multiRP_single_top[DataSet_multiRP_single_top['class_predict8']==1]['weight'])
VV_inclusive_counting_8 = np.sum(DataSet_multiRP_VV_inclusivo[DataSet_multiRP_VV_inclusivo['class_predict8']==1]['weight'])
WJets_counting_8 = np.sum(DataSet_multiRP_WJets[DataSet_multiRP_WJets['class_predict8']==1]['weight'])
ttbar_counting_8 = np.sum(DataSet_multiRP_TTbar[DataSet_multiRP_TTbar['class_predict8']==1]['weight'])
ANOMALO8_counting_8 = np.sum(ANOMALO8[ANOMALO8['class_predict']==1]['weight'])
SM_counting_8 = np.sum(SM[SM['class_predict8']==1]['weight'])

DrellYan_counting_7 = np.sum(DataSet_multiRP_DrellYan[DataSet_multiRP_DrellYan['class_predict7']==1]['weight'])
QCD_counting_7 = np.sum(DataSet_multiRP_QCD[DataSet_multiRP_QCD['class_predict7']==1]['weight'])
Single_top_counting_7 = np.sum(DataSet_multiRP_single_top[DataSet_multiRP_single_top['class_predict7']==1]['weight'])
VV_inclusive_counting_7 = np.sum(DataSet_multiRP_VV_inclusivo[DataSet_multiRP_VV_inclusivo['class_predict7']==1]['weight'])
WJets_counting_7 = np.sum(DataSet_multiRP_WJets[DataSet_multiRP_WJets['class_predict7']==1]['weight'])
ttbar_counting_7 = np.sum(DataSet_multiRP_TTbar[DataSet_multiRP_TTbar['class_predict7']==1]['weight'])
ANOMALO7_counting_7 = np.sum(ANOMALO7[ANOMALO7['class_predict']==1]['weight'])
SM_counting_7 = np.sum(SM[SM['class_predict7']==1]['weight'])

DrellYan_counting_6 = np.sum(DataSet_multiRP_DrellYan[DataSet_multiRP_DrellYan['class_predict6']==1]['weight'])
QCD_counting_6 = np.sum(DataSet_multiRP_QCD[DataSet_multiRP_QCD['class_predict6']==1]['weight'])
Single_top_counting_6 = np.sum(DataSet_multiRP_single_top[DataSet_multiRP_single_top['class_predict6']==1]['weight'])
VV_inclusive_counting_6 = np.sum(DataSet_multiRP_VV_inclusivo[DataSet_multiRP_VV_inclusivo['class_predict6']==1]['weight'])
WJets_counting_6 = np.sum(DataSet_multiRP_WJets[DataSet_multiRP_WJets['class_predict6']==1]['weight'])
ttbar_counting_6 = np.sum(DataSet_multiRP_TTbar[DataSet_multiRP_TTbar['class_predict6']==1]['weight'])
ANOMALO6_counting_6 = np.sum(ANOMALO6[ANOMALO6['class_predict']==1]['weight'])
SM_counting_6 = np.sum(SM[SM['class_predict6']==1]['weight'])

DrellYan_counting_5 = np.sum(DataSet_multiRP_DrellYan[DataSet_multiRP_DrellYan['class_predict5']==1]['weight'])
QCD_counting_5 = np.sum(DataSet_multiRP_QCD[DataSet_multiRP_QCD['class_predict5']==1]['weight'])
Single_top_counting_5 = np.sum(DataSet_multiRP_single_top[DataSet_multiRP_single_top['class_predict5']==1]['weight'])
VV_inclusive_counting_5 = np.sum(DataSet_multiRP_VV_inclusivo[DataSet_multiRP_VV_inclusivo['class_predict5']==1]['weight'])
WJets_counting_5 = np.sum(DataSet_multiRP_WJets[DataSet_multiRP_WJets['class_predict7']==1]['weight'])
ttbar_counting_5 = np.sum(DataSet_multiRP_TTbar[DataSet_multiRP_TTbar['class_predict5']==1]['weight'])
ANOMALO5_counting_5 = np.sum(ANOMALO5[ANOMALO5['class_predict']==1]['weight'])
SM_counting_5 = np.sum(SM[SM['class_predict5']==1]['weight'])

DrellYan_counting_1 = np.sum(DataSet_multiRP_DrellYan[DataSet_multiRP_DrellYan['class_predict1']==1]['weight'])
QCD_counting_1 = np.sum(DataSet_multiRP_QCD[DataSet_multiRP_QCD['class_predict1']==1]['weight'])
Single_top_counting_1 = np.sum(DataSet_multiRP_single_top[DataSet_multiRP_single_top['class_predict1']==1]['weight'])
VV_inclusive_counting_1 = np.sum(DataSet_multiRP_VV_inclusivo[DataSet_multiRP_VV_inclusivo['class_predict1']==1]['weight'])
WJets_counting_1 = np.sum(DataSet_multiRP_WJets[DataSet_multiRP_WJets['class_predict1']==1]['weight'])
ttbar_counting_1 = np.sum(DataSet_multiRP_TTbar[DataSet_multiRP_TTbar['class_predict1']==1]['weight'])
ANOMALO7_counting_1 = np.sum(ANOMALO1[ANOMALO1['class_predict']==1]['weight'])
SM_counting_1 = np.sum(SM[SM['class_predict1']==1]['weight'])

DrellYan_counting_2 = np.sum(DataSet_multiRP_DrellYan[DataSet_multiRP_DrellYan['class_predict2']==1]['weight'])
QCD_counting_2 = np.sum(DataSet_multiRP_QCD[DataSet_multiRP_QCD['class_predict2']==1]['weight'])
Single_top_counting_2 = np.sum(DataSet_multiRP_single_top[DataSet_multiRP_single_top['class_predict2']==1]['weight'])
VV_inclusive_counting_2 = np.sum(DataSet_multiRP_VV_inclusivo[DataSet_multiRP_VV_inclusivo['class_predict2']==1]['weight'])
WJets_counting_2 = np.sum(DataSet_multiRP_WJets[DataSet_multiRP_WJets['class_predict2']==1]['weight'])
ttbar_counting_2 = np.sum(DataSet_multiRP_TTbar[DataSet_multiRP_TTbar['class_predict2']==1]['weight'])
ANOMALO2_counting_2 = np.sum(ANOMALO2[ANOMALO2['class_predict']==1]['weight'])
SM_counting_2 = np.sum(SM[SM['class_predict2']==1]['weight'])

DrellYan_counting_3 = np.sum(DataSet_multiRP_DrellYan[DataSet_multiRP_DrellYan['class_predict3']==1]['weight'])
QCD_counting_3 = np.sum(DataSet_multiRP_QCD[DataSet_multiRP_QCD['class_predict3']==1]['weight'])
Single_top_counting_3 = np.sum(DataSet_multiRP_single_top[DataSet_multiRP_single_top['class_predict3']==1]['weight'])
VV_inclusive_counting_3 = np.sum(DataSet_multiRP_VV_inclusivo[DataSet_multiRP_VV_inclusivo['class_predict3']==1]['weight'])
WJets_counting_3 = np.sum(DataSet_multiRP_WJets[DataSet_multiRP_WJets['class_predict3']==1]['weight'])
ttbar_counting_3 = np.sum(DataSet_multiRP_TTbar[DataSet_multiRP_TTbar['class_predict3']==1]['weight'])
ANOMALO7_counting_3 = np.sum(ANOMALO3[ANOMALO3['class_predict']==1]['weight'])
SM_counting_3 = np.sum(SM[SM['class_predict3']==1]['weight'])

DrellYan_counting_4 = np.sum(DataSet_multiRP_DrellYan[DataSet_multiRP_DrellYan['class_predict4']==1]['weight'])
QCD_counting_4 = np.sum(DataSet_multiRP_QCD[DataSet_multiRP_QCD['class_predict4']==1]['weight'])
Single_top_counting_4 = np.sum(DataSet_multiRP_single_top[DataSet_multiRP_single_top['class_predict4']==1]['weight'])
VV_inclusive_counting_4 = np.sum(DataSet_multiRP_VV_inclusivo[DataSet_multiRP_VV_inclusivo['class_predict4']==1]['weight'])
WJets_counting_4 = np.sum(DataSet_multiRP_WJets[DataSet_multiRP_WJets['class_predict4']==1]['weight'])
ttbar_counting_4 = np.sum(DataSet_multiRP_TTbar[DataSet_multiRP_TTbar['class_predict4']==1]['weight'])
ANOMALO4_counting_4 = np.sum(ANOMALO4[ANOMALO4['class_predict']==1]['weight'])
SM_counting_4 = np.sum(SM[SM['class_predict4']==1]['weight'])

# Datacard

* ### O layout do datacard é o seguinte:


No topo estão os números imax, jmax e kmax que representam o número de bins, processos e parâmetros de incômodo(nuisance), respectivamente.
Aqui, um "bin" pode se referir a uma contagem literal de evento único, ou uma distribuição completa que estamos ajustando, em geral com muitos bins de histograma. É possível substituir esses números por * e eles serão deduzidos automaticamente.

A primeira linha começando com bin fornece um rótulo exclusivo para cada canal, e a linha seguinte começando com observação fornece o número de eventos observados dos $\textbf{DADOS}$.

As primeiras quatro linhas rotuladas bin, process, process e rate fornecem o rótulo do canal, o rótulo do processo, um identificador de processo (<= 0 para signal,> 0 para background) e o número de eventos esperados, respectivamente.

As linhas restantes descrevem fontes de incerteza sistemática. Cada linha dá o nome da incerteza (que se tornará o nome do parâmetro incômodo dentro de nosso modelo RooFit), o tipo de incerteza ("lnN" = incerteza de normalização logarítmica) e o efeito em cada processo em cada canal. Por exemplo. uma incerteza de 20% no rendimento é escrita como 1,20.

Também é possível adicionar um símbolo hash (#) no início de uma linha, que a combinação irá ignorar quando ler o cartão.

* ### Limites assintóticos


Como estamos procurando por um processo de sinal que não existe no modelo padrão, é natural definir um limite superior na seção de choque vezes a fração de ramificação do processo (assumindo que nosso conjunto de dados não contém uma descoberta significativa de nova física). $\texttt{Combine}$ tem método dedicado para calcular os limites superiores. O mais comumente usado é $\texttt{AsymptoticLimits}$, que implementa o critério $\texttt{CLs}$ e usa a razão de verossimilhança do perfil como estatística de teste. Como o nome indica, as distribuições estatísticas de teste são determinadas analiticamente na aproximação assintótica, portanto, não há necessidade de mais tempo para arremessar e ajustar o brinquedo. Executando o seguinte comando: 

$\texttt{combine -M AsymptoticLimits datacard_part1.txt -n .part1A}$

Você deve ver os resultados dos cálculos dos limites observados e esperados impressos na tela. Aqui adicionamos uma opção extra, -n .part1A, que é a abreviação de --name, e é usada para rotular os arquivos de saída combinados produzidos.

In [ ]:
print('Inciando o Datacard com o método simple_counting\n')

dc=open('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alpha0_5e-6.txt','w')

dc.write('imax *  number of channels\n')
dc.write('jmax *  number of processes -1\n')
dc.write('kmax *  number of nuisance parameters (sources of systematical uncertainties)\n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the data observations
dc.write('bin           signal_region  \n')
dc.write('observation       {}\n'.format(len( n_eventos_Data_after_cut8 )))

dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the expectations for signal and background
dc.write('bin       signal_region    signal_region      signal_region    \n')
dc.write('process     Anomalo8       Standard_Model      Background        \n')
dc.write('process         0                 1                2            \n')
dc.write('rate         {0:.3f}             {1:.3f}             {2:.3f}    \n'.format(DataSet_Test8_weight_signal[ n_events_signal_after_cut8 ].sum() / test_size,
                                                                                     DataSet_TestSM8_weight_signal[ n_events_SM_after_cut8 ].sum() / test_size,
                                                                                     DataSet_Test8_weight_backgr[ n_events_back_after_cut8 ].sum() / test_size
                                                                                                                                                     ))
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the sources of systematic uncertainty
dc.write('lumi      lnN  1.025                1.0              1.0         \n')
#dc.write('effic_pps lnN  1.10                 -               -       \n')


       
#all done  cross_section_TT
dc.close()

print('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard.dat is ready to be used with combine\n\n')

import os

_ = os.system('cat /content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alpha0_5e-6.txt')

In [ ]:
print('Inciando o Datacard com o método simple_counting\n')

dc=open('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alpha0_2e-6.txt','w')

dc.write('imax *  number of channels\n')
dc.write('jmax *  number of processes -1\n')
dc.write('kmax *  number of nuisance parameters (sources of systematical uncertainties)\n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the data observations
dc.write('bin           signal_region  \n')
dc.write('observation       {}\n'.format(len( n_eventos_Data_after_cut7 )))

dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the expectations for signal and background
dc.write('bin       signal_region    signal_region      signal_region     \n')
dc.write('process     Anomalo7       Standard_Model       Background         \n')
dc.write('process         0                 1                2           \n')
dc.write('rate         {0:.3f}             {1:.3f}             {2:.3f}     \n'.format(DataSet_Test7_weight_signal[ n_events_signal_after_cut7 ].sum() / test_size ,
                                                                                      DataSet_TestSM7_weight_signal[ n_events_SM_after_cut7 ].sum() / test_size,
                                                                                      DataSet_Test7_weight_backgr[ n_events_back_after_cut7 ].sum() / test_size
                                                                               
                                                                                                                                                     ))
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the sources of systematic uncertainty
dc.write('lumi      lnN  1.025               1.0               1.0      \n')
#dc.write('effic_pps lnN  1.10              1.10             1.10    \n')


       
#all done  cross_section_TT
dc.close()

print('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard.dat is ready to be used with combine\n\n')

import os

_ = os.system('cat /content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alpha0_2e-6.txt')

In [ ]:
print('Inciando o Datacard com o método simple_counting\n')

dc=open('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alpha0_1e-6.txt','w')

dc.write('imax *  number of channels\n')
dc.write('jmax *  number of processes -1\n')
dc.write('kmax *  number of nuisance parameters (sources of systematical uncertainties)\n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the data observations
dc.write('bin           signal_region  \n')
dc.write('observation       {}\n'.format(len( n_eventos_Data_after_cut6 )))

dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the expectations for signal and background
dc.write('bin       signal_region    signal_region      signal_region    \n')
dc.write('process     Anomalo6      Standard_Model       Background       \n')
dc.write('process         0                 1                2           \n')
dc.write('rate         {0:.3f}             {1:.3f}             {2:.3f}   \n'.format(DataSet_Test6_weight_signal[ n_events_signal_after_cut6].sum() / test_size,
                                                                                    DataSet_TestSM6_weight_signal[ n_events_SM_after_cut6 ].sum() / test_size,
                                                                                    DataSet_Test6_weight_backgr[ n_events_back_after_cut6 ].sum() / test_size
                                                                            ,
                                                                                                                                                     ))
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the sources of systematic uncertainty
dc.write('lumi      lnN  1.025                1.0              1.0humm          \n')
#dc.write('effic_pps lnN  1.10              1.10             1.10          \n')


       
#all done  cross_section_TT
dc.close()

print('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard.dat is ready to be used with combine\n\n')

import os

_ = os.system('cat /content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alpha0_1e-6.txt')

In [ ]:
print('Inciando o Datacard com o método simple_counting\n')

dc=open('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alpha0_0.5e-6.txt','w')

dc.write('imax *  number of channels\n')
dc.write('jmax *  number of processes -1\n')
dc.write('kmax *  number of nuisance parameters (sources of systematical uncertainties)\n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the data observations
dc.write('bin           signal_region  \n')
dc.write('observation       {}\n'.format(len( n_eventos_Data_after_cut5 )))

dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the expectations for signal and background
dc.write('bin       signal_region    signal_region      signal_region       \n')
dc.write('process     Anomalo5       Standard_Model      Background         \n')
dc.write('process         0                 1                2              \n')
dc.write('rate         {0:.3f}             {1:.3f}         {2:.3f}                                                                             \n'.format(DataSet_Test5_weight_signal[ n_events_signal_after_cut5 ].sum() / test_size,
                                                                                                                                                   DataSet_TestSM5_weight_signal[ n_events_SM_after_cut5 ].sum() / test_size,
                                                                                                                                                   DataSet_Test5_weight_backgr[ n_events_back_after_cut5 ].sum() / test_size
                                                                                                                                                     ))
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the sources of systematic uncertainty
dc.write('lumi      lnN  1.025                -               -                 \n')
#dc.write('xsecttbar lnN    -                 -              1.013              \n')
dc.write('effic_pps lnN  1.10              1.10             1.10                \n')


       
#all done  cross_section_TT
dc.close()

print('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard.dat is ready to be used with combine\n\n')

import os

_ = os.system('cat /content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alpha0_0.5e-6.txt')

In [ ]:
dc

In [ ]:
print('Inciando o Datacard com o método simple_counting\n')

dc=open('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alphaC_8e-6.txt','w')

dc.write('imax *  number of channels\n')
dc.write('jmax *  number of processes -1\n')
dc.write('kmax *  number of nuisance parameters (sources of systematical uncertainties)\n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the data observations
dc.write('bin           signal_region  \n')
dc.write('observation       {}\n'.format(len( n_eventos_Data_after_cut4 )))

dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the expectations for signal and background
dc.write('bin       signal_region    signal_region      signal_region    \n')
dc.write('process     Anomalo4       Standard_Model       Background       \n')
dc.write('process         0                 1                2           \n')
dc.write('rate         {0:.3f}               {1:.3f}         {2:.3f}   \n'.format(DataSet_Test4_weight_signal[ n_events_signal_after_cut4 ].sum() / test_size,
                                                                                DataSet_TestSM4_weight_signal[ n_events_SM_after_cut4 ].sum() / test_size,
                                                                                DataSet_Test4_weight_backgr[ n_events_back_after_cut4 ].sum() / test_size,
                                                                                                                                                     ))
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the sources of systematic uncertainty
dc.write('lumi      lnN  1.025             1.025            1.025     \n')
#dc.write('xsecttbar lnN    -                 -              1.013  \n')
dc.write('effic_pps lnN  1.10              1.10             1.10    \n')


       
#all done  cross_section_TT
dc.close()

print('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard.dat is ready to be used with combine\n\n')

import os

_ = os.system('cat /content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alphaC_8e-6.txt')

In [ ]:
print('Inciando o Datacard com o método simple_counting\n')

dc=open('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alphaC_2e-5.txt','w')

dc.write('imax *  number of channels\n')
dc.write('jmax *  number of processes -1\n')
dc.write('kmax *  number of nuisance parameters (sources of systematical uncertainties)\n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the data observations
dc.write('bin           signal_region  \n')
dc.write('observation       {}\n'.format(len( n_eventos_Data_after_cut2 )))

dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the expectations for signal and background
dc.write('bin       signal_region    signal_region      signal_region    \n')
dc.write('process     Anomalo1       Standard_Model       Background       \n')
dc.write('process         0                 1                2           \n')
dc.write('rate         {0:.3f}             {1:.3f}         {2:.3f}   \n'.format(DataSet_Test1_weight_signal[ n_events_signal_after_cut1 ].sum() / test_size,
                                                                                DataSet_TestSM1_weight_signal[ n_events_SM_after_cut1 ].sum() / test_size,
                                                                                DataSet_Test1_weight_backgr[ n_events_back_after_cut1 ].sum() / test_size,
                                                                                                                                                     ))
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the sources of systematic uncertainty
dc.write('lumi      lnN  1.025                -               -     \n')
#dc.write('xsecttbar lnN    -                 -              1.013  \n')
dc.write('effic_pps lnN  1.10              1.10             1.10    \n')


       
#all done  cross_section_TT
dc.close()

print('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard.dat is ready to be used with combine\n\n')

import os

_ = os.system('cat /content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alphaC_2e-5.txt')

In [ ]:
print('Inciando o Datacard com o método simple_counting\n')

dc=open('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alphaC_5e-6.txt','w')

dc.write('imax *  number of channels\n')
dc.write('jmax *  number of processes -1\n')
dc.write('kmax *  number of nuisance parameters (sources of systematical uncertainties)\n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the data observations
dc.write('bin           signal_region  \n')
dc.write('observation       {}\n'.format(len( n_eventos_Data_after_cut3 )))

dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the expectations for signal and background
dc.write('bin       signal_region    signal_region      signal_region    \n')
dc.write('process     Anomalo3       Standard_Model       Background       \n')
dc.write('process         0                 1                2           \n')
dc.write('rate         {0:.3f}             {1:.3f}         {2:.3f}   \n'.format(DataSet_Test3_weight_signal[ n_events_signal_after_cut3 ].sum() / test_size,
                                                                                DataSet_TestSM3_weight_signal[ n_events_SM_after_cut3 ].sum() / test_size,
                                                                                DataSet_Test3_weight_backgr[ n_events_back_after_cut3 ].sum() / test_size,
                                                                                                                                                     ))
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the sources of systematic uncertainty
dc.write('lumi      lnN  1.025                -               -     \n')
#dc.write('xsecttbar lnN    -                 -              1.013  \n')
dc.write('effic_pps lnN  1.10              1.10             1.10    \n')


       
#all done  cross_section_TT
dc.close()

print('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard.dat is ready to be used with combine\n\n')

import os

_ = os.system('cat /content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alphaC_5e-6.txt')

In [ ]:
print('Inciando o Datacard com o método simple_counting\n')

dc=open('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alphaC_2e-6.txt','w')

dc.write('imax *  number of channels\n')
dc.write('jmax *  number of processes -1\n')
dc.write('kmax *  number of nuisance parameters (sources of systematical uncertainties)\n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the data observations
dc.write('bin           signal_region  \n')
dc.write('observation       {}\n'.format(len( n_eventos_Data_after_cut2 )))

dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the expectations for signal and background
dc.write('bin       signal_region    signal_region      signal_region    \n')
dc.write('process     Anomalo2       Standard_Model       Background       \n')
dc.write('process         0                 1                2           \n')
dc.write('rate         {0:.3f}             {1:.3f}             {2:.3f}   \n'.format(DataSet_Test2_weight_signal[ n_events_signal_after_cut2 ].sum() / test_size,
                                                                                DataSet_TestSM2_weight_signal[ n_events_SM_after_cut2 ].sum() / test_size,
                                                                                DataSet_Test2_weight_backgr[ n_events_back_after_cut2 ].sum() / test_size,
                                                                                                                                                     ))
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')
dc.write('----------------------------------------------------------------------------------------------------------------------------------------------------- \n')

#add the sources of systematic uncertainty
dc.write('lumi      lnN  1.025                -               -     \n')
#dc.write('xsecttbar lnN    -                 -              1.013  \n')
dc.write('effic_pps lnN  1.10              1.10             1.10    \n')


       
#all done  cross_section_TT
dc.close()

print('/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard.dat is ready to be used with combine\n\n')

import os

_ = os.system('cat /content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/datacard_simple_counting_alphaC_2e-6.txt')

# Inspecionando o arquivo de shape

As linhas de código a seguir abrem o arquivo de formas e fazem uma comparação rápida de dados/expectativas antes de qualquer ajuste. 

In [ ]:
def plots_results( var, label_x, range_ ):
    
    fig, ax = plt.subplots( 2,2, figsize=(19,13) )
    var = var
    label_x = label_x
    range_ = range_
    bins = 15
    
    data_set_dados_multirp8 = data_set_dados_multirp[data_set_dados_multirp[ 'class_predict8' ] == 1 ]
    data_set_dados_multirp7 = data_set_dados_multirp[data_set_dados_multirp[ 'class_predict7' ] == 1 ]
    data_set_dados_multirp1 = data_set_dados_multirp[data_set_dados_multirp[ 'class_predict1' ] == 1 ]
    data_set_dados_multirp4 = data_set_dados_multirp[data_set_dados_multirp[ 'class_predict4' ] == 1 ]
    
    counts_8, bin_edges_8 = np.histogram( data_set_dados_multirp8[var] , bins = bins, range = range_ )
    errors_8 = np.sqrt( counts_8 )
    bin_centres_8 = ( bin_edges_8[:-1] + bin_edges_8[1:] ) / 2.

    counts_7, bin_edges_7 = np.histogram( data_set_dados_multirp7[var] , bins = bins, range = range_ )
    errors_7 = np.sqrt( counts_7 )
    bin_centres_7 = ( bin_edges_7[:-1] + bin_edges_7[1:] ) / 2.
    
    counts_1, bin_edges_1 = np.histogram( data_set_dados_multirp1[var] , bins = bins, range = range_ )
    errors_1 = np.sqrt( counts_1 )
    bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
    
    counts_4, bin_edges_4 = np.histogram( data_set_dados_multirp4[var] , bins = bins, range = range_ )
    errors_4 = np.sqrt( counts_4 )
    bin_centres_4 = ( bin_edges_4[:-1] + bin_edges_4[1:] ) / 2.    
    
    num = 8
    df_background8 = [ DataSet_multiRP_DrellYan[ DataSet_multiRP_DrellYan['class_predict{}'.format(num)] == 1 ][var], 
                      DataSet_multiRP_QCD[ DataSet_multiRP_QCD['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_single_top[ DataSet_multiRP_single_top['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_TTbar[ DataSet_multiRP_TTbar['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_VV_inclusivo[ DataSet_multiRP_VV_inclusivo['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_WJets[ DataSet_multiRP_WJets['class_predict{}'.format(num)] == 1 ][var] ]

    df_background_weight8 = [ DataSet_multiRP_DrellYan[ DataSet_multiRP_DrellYan['class_predict{}'.format(num)] == 1 ]['weight'], 
                      DataSet_multiRP_QCD[ DataSet_multiRP_QCD['class_predict{}'.format(num)] == 1 ]['weight'] ,
                      DataSet_multiRP_single_top[ DataSet_multiRP_single_top['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_TTbar[ DataSet_multiRP_TTbar['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_VV_inclusivo[ DataSet_multiRP_VV_inclusivo['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_WJets[ DataSet_multiRP_WJets['class_predict{}'.format(num)] == 1 ]['weight'] ]
    num = 7
    df_background7 = [ DataSet_multiRP_DrellYan[ DataSet_multiRP_DrellYan['class_predict{}'.format(num)] == 1 ][var], 
                      DataSet_multiRP_QCD[ DataSet_multiRP_QCD['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_single_top[ DataSet_multiRP_single_top['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_TTbar[ DataSet_multiRP_TTbar['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_VV_inclusivo[ DataSet_multiRP_VV_inclusivo['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_WJets[ DataSet_multiRP_WJets['class_predict{}'.format(num)] == 1 ][var] ]

    df_background_weight7 = [ DataSet_multiRP_DrellYan[ DataSet_multiRP_DrellYan['class_predict{}'.format(num)] == 1 ]['weight'], 
                      DataSet_multiRP_QCD[ DataSet_multiRP_QCD['class_predict{}'.format(num)] == 1 ]['weight'] ,
                      DataSet_multiRP_single_top[ DataSet_multiRP_single_top['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_TTbar[ DataSet_multiRP_TTbar['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_VV_inclusivo[ DataSet_multiRP_VV_inclusivo['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_WJets[ DataSet_multiRP_WJets['class_predict{}'.format(num)] == 1 ]['weight'] ]

    num = 1
    df_background1 = [ DataSet_multiRP_DrellYan[ DataSet_multiRP_DrellYan['class_predict{}'.format(num)] == 1 ][var], 
                      DataSet_multiRP_QCD[ DataSet_multiRP_QCD['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_single_top[ DataSet_multiRP_single_top['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_TTbar[ DataSet_multiRP_TTbar['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_VV_inclusivo[ DataSet_multiRP_VV_inclusivo['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_WJets[ DataSet_multiRP_WJets['class_predict{}'.format(num)] == 1 ][var] ]

    df_background_weight1 = [ DataSet_multiRP_DrellYan[ DataSet_multiRP_DrellYan['class_predict{}'.format(num)] == 1 ]['weight'], 
                      DataSet_multiRP_QCD[ DataSet_multiRP_QCD['class_predict{}'.format(num)] == 1 ]['weight'] ,
                      DataSet_multiRP_single_top[ DataSet_multiRP_single_top['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_TTbar[ DataSet_multiRP_TTbar['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_VV_inclusivo[ DataSet_multiRP_VV_inclusivo['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_WJets[ DataSet_multiRP_WJets['class_predict{}'.format(num)] == 1 ]['weight'] ]

    num = 4
    df_background4 = [ DataSet_multiRP_DrellYan[ DataSet_multiRP_DrellYan['class_predict{}'.format(num)] == 1 ][var], 
                      DataSet_multiRP_QCD[ DataSet_multiRP_QCD['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_single_top[ DataSet_multiRP_single_top['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_TTbar[ DataSet_multiRP_TTbar['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_VV_inclusivo[ DataSet_multiRP_VV_inclusivo['class_predict{}'.format(num)] == 1 ][var],
                      DataSet_multiRP_WJets[ DataSet_multiRP_WJets['class_predict{}'.format(num)] == 1 ][var] ]

    df_background_weight4 = [ DataSet_multiRP_DrellYan[ DataSet_multiRP_DrellYan['class_predict{}'.format(num)] == 1 ]['weight'], 
                      DataSet_multiRP_QCD [ DataSet_multiRP_QCD['class_predict{}'.format(num)] == 1 ]['weight'] ,
                      DataSet_multiRP_single_top[ DataSet_multiRP_single_top['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_TTbar[ DataSet_multiRP_TTbar['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_VV_inclusivo[ DataSet_multiRP_VV_inclusivo['class_predict{}'.format(num)] == 1 ]['weight'],
                      DataSet_multiRP_WJets[ DataSet_multiRP_WJets['class_predict{}'.format(num)] == 1 ]['weight'] ]



    label_back = [ 'Drell-Yan', 'QCD', 'Single Top', r'$t\bar{t}$', '(WW,WZ,ZZ) Inclusive', 'W+Jets' ]

    ax[0,0].hist( df_background8, stacked = True, bins = bins, range = range_, histtype = 'bar', label = label_back, weights = df_background_weight8 )
    ax[0,0].hist( ANOMALO8[ANOMALO8['class_predict'] == 1 ][var], ls=':', linewidth=3, bins = bins, range = range_, histtype = 'step', color = 'blue', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5 \times 10^{-6} $', weights = ANOMALO8[ANOMALO8['class_predict'] == 1 ]['weight'] )
    #ax[0,0].errorbar( bin_centres_8, counts_8, yerr=errors_8, xerr=abs(bin_centres_8[0] - bin_centres_8[1])/2,fmt='.', label = 'Data-2016', color = 'black' )
    ax[0,0].legend(loc='best', fontsize = 12)
    ax[0,0].set_xlabel(label_x, fontsize = 15)
    ax[0,0].set_ylabel('Events', fontsize = 15)

    ax[1,0].hist( df_background7, stacked = True, bins = bins, range = range_, histtype = 'bar', label = label_back, weights = df_background_weight7 )
    ax[1,0].hist( ANOMALO7[ANOMALO7['class_predict'] == 1 ][var], ls=':', linewidth=3, bins = bins, range = range_, histtype = 'step', color = 'blue', label =  r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', weights = ANOMALO7[ANOMALO7['class_predict'] == 1 ]['weight'] )
    #ax[1,0].errorbar( bin_centres_7, counts_7, yerr=errors_7, xerr=abs(bin_centres_7[0] - bin_centres_7[1])/2,fmt='.', label = 'Data-2016', color = 'black' )    
    ax[1,0].legend(loc='best', fontsize = 12)
    ax[1,0].set_xlabel(label_x, fontsize = 15)
    ax[1,0].set_ylabel('Events', fontsize = 15)

    ax[0,1].hist( df_background1, stacked = True, bins = bins, range = range_, histtype = 'bar', label = label_back, weights = df_background_weight1 )
    ax[0,1].hist( ANOMALO1[ANOMALO1['class_predict'] == 1 ][var], ls=':', linewidth=3, bins = bins, range = range_, histtype = 'step', color = 'blue', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=2.0 \times 10^{-5}$', weights = ANOMALO1[ANOMALO1['class_predict'] == 1 ]['weight'] )
    #ax[0,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[0] - bin_centres_1[1])/2,fmt='.', label = 'Data-2016', color = 'black' )    
    ax[0,1].legend(loc='best', fontsize = 12)
    ax[0,1].set_xlabel(label_x, fontsize = 15)
    ax[0,1].set_ylabel('Events', fontsize = 15)

    ax[1,1].hist( df_background4, stacked = True, bins = bins, range = range_, histtype = 'bar', label = label_back, weights = df_background_weight4 )
    ax[1,1].hist( ANOMALO4[ANOMALO4['class_predict'] == 1 ][var], ls=':', linewidth=3, bins = bins, range = range_, histtype = 'step', color = 'blue', label =  r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=8 \times 10^{-6}$', weights = ANOMALO4[ANOMALO4['class_predict'] == 1 ]['weight'] )
    #ax[1,1].errorbar( bin_centres_4, counts_4, yerr=errors_4, xerr=abs(bin_centres_4[0] - bin_centres_4[1])/2,fmt='.', label = 'Data-2016', color = 'black' )    
    ax[1,1].legend(loc='best', fontsize = 12)
    ax[1,1].set_xlabel(label_x, fontsize = 15)
    ax[1,1].set_ylabel('Events', fontsize = 15)

# Histogramas das variáveis após o treinamento e corte no discriminante

In [ ]:
plots_results('Yww_Yx','Yww - Yx', (-2.5,2.5))

In [ ]:
plots_results('Mww/Mx','$\dfrac{Mww}{M_{PPS}}$', (0,2))

In [ ]:
plots_results('ExtraTracks', 'ExtraTracks', (0,10))

In [ ]:
plots_results('METPt','$P_{T}^{MET}$', (80,500))

In [ ]:
plots_results('jetAK8_prunedMass','$PrunedMass^{JetAK8}$', (0,200))

In [ ]:
plots_results('jetAK8_pt','$P_{T}^{JetAK8}$', (200,1000))

In [ ]:
plots_results('Pt_W_lep', '$P_{T}^{W_{lep}}$', (200,1000))

In [ ]:
plots_results('Mww','$M_{WW}$', (500,1000))

In [ ]:
plots_results('muon_pt','$\mu_{pt}$',(100,600))

# Usando o Smearing no resultado do treinamento

In [ ]:
from random import gauss

def xi_smearing(dset):
    df_xi1 = []
    df_xi2 = []
    for i in range(0,len(DataSet_Test5_)):
        df_xi1.append( gauss( 0, 0.1 * dset['xi1'][i] ) + np.array( dset['xi1'][i] ) )
        df_xi2.append( gauss( 0, 0.1 * dset['xi2'][i] ) + np.array( dset['xi2'][i] ) )
    xi1_smearing = pd.DataFrame( df_xi1, columns = ['xi1_smearing'] ) 
    xi2_smearing = pd.DataFrame( df_xi2, columns = ['xi2_smearing'] ) 
    return pd.concat( [ xi1_smearing, xi2_smearing ], axis = 1 )

In [ ]:
select_columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass', 'jetAK8_tau21', 'METPt', 
                  'muon_pt', 'muon_eta', 'ExtraTracks', 'Yww', 'xi1_smearing', 'xi2_smearing', 'Mx', 'Yx', 'Mww/Mx', 'Yww_Yx']

In [ ]:
from tqdm import tqdm # mostra o tempo que passou e quanto tempo falta para terminar o loop
def smearing_loop( dset, n_interation, bst, label_ ):
    select_columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass', 
                      'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'Yww', 'xi1', 'xi2', 'Mx', 'Yx', 'Mww/Mx', 'Yww_Yx']
    n_events_smearing_Signal = []
    n_events_Signal = []
    for i in tqdm(range(n_interation)):
        if i == 0: 
            dset_SM_mod = dset[dset['label']==2]
        df_xi1_SM = []
        df_xi2_SM = []
        for j in range( 0, len( dset[ dset[ 'label' ] == 2 ] ) ):
            df_xi1_SM.append( gauss( 0, 0.1 * np.array(dset[dset['label']==2]['xi1'])[j] ) + np.array(dset[dset['label']==2]['xi1'])[j] ) 
            df_xi2_SM.append( gauss( 0, 0.1 * np.array(dset[dset['label']==2]['xi2'])[j] ) + np.array(dset[dset['label']==2]['xi2'])[j] )
        dset_SM_mod['xi1'] = df_xi1_SM
        dset_SM_mod['xi2'] = df_xi2_SM
        dset_SM_mod['Mx'] = 13000 * ( dset_SM_mod['xi1'] * dset_SM_mod['xi2'] )**0.5
        dset_SM_mod['Yx'] = 0.5 * ( np.log( dset_SM_mod['xi1'] / dset_SM_mod['xi2'] ) )
        dset_SM_mod['Mww/Mx'] = dset_SM_mod['Mww'] / dset_SM_mod['Mx'] 
        dset_SM_mod['Yww_Yx'] = dset_SM_mod['Yww'] - dset_SM_mod['Yx']

        if i == 0: 
            dset_ANOMALO_mod = dset[ dset[ 'label' ] == 1 ]
        df_xi1_ANOMALO = []
        df_xi2_ANOMALO = []
        for j in range( 0, len( dset[ dset[ 'label' ] == 1 ] ) ):
            df_xi1_ANOMALO.append( gauss( 0, 0.1 * np.array(dset[dset['label']==1]['xi1'])[j] ) + np.array(dset[dset['label']==1]['xi1'])[j] ) 
            df_xi2_ANOMALO.append( gauss( 0, 0.1 * np.array(dset[dset['label']==1]['xi2'])[j] ) + np.array(dset[dset['label']==1]['xi2'])[j] )

        dset_ANOMALO_mod['xi1'] = df_xi1_ANOMALO
        dset_ANOMALO_mod['xi2'] = df_xi2_ANOMALO
        dset_ANOMALO_mod['Mx'] = 13000 * ( dset_ANOMALO_mod['xi1'] * dset_ANOMALO_mod['xi2'] )**0.5
        dset_ANOMALO_mod['Yx'] = 0.5 * ( np.log( dset_ANOMALO_mod['xi1'] / dset_ANOMALO_mod['xi2'] ) )
        dset_ANOMALO_mod['Mww/Mx'] = dset_ANOMALO_mod['Mww'] / dset_ANOMALO_mod['Mx'] 
        dset_ANOMALO_mod['Yww_Yx'] = dset_ANOMALO_mod['Yww'] - dset_ANOMALO_mod['Yx']
        
        dset_back_mod = dset[ dset[ 'label' ] == 0 ] 
        
        dset_back_mod['xi1'] = dset[dset['label']==0]['xi1']
        dset_back_mod['xi2'] = dset[dset['label']==0]['xi2']

        dset_full = pd.concat( [ dset_SM_mod, dset_ANOMALO_mod, dset_back_mod ], axis = 0 )    
        y_test = dset_full.label
        predict_proba = bst.predict(dset_full[select_columns])

        n_events_back_after_cut = predict_proba[ y_test == 0 ][:,1] > best_cut( y_test, predict_proba ) 
        n_events_signal_after_cut = predict_proba[ y_test == 1 ][:,1] > best_cut( y_test,predict_proba )
        n_events_SM_after_cut = predict_proba[ y_test == 2 ][:,1] > best_cut( y_test,predict_proba )
        print('Numero de eventos de background depois do corte COM smearing -->', dset_full[dset_full['label']==0]['weight'][ n_events_back_after_cut ].sum() / test_size )
        print('Numero de eventos de signal depois do corte COM smearing -->', dset_full[dset_full['label']==1]['weight'][ n_events_signal_after_cut ].sum() / test_size )
        print('Numero de eventos de SM depois do corte COM smearing -->', dset_full[dset_full['label']==2]['weight'][ n_events_SM_after_cut ].sum() / test_size )
        '''
        dataset_sem_smearing = dset

        predict_proba_SEM = bst.predict(dataset_sem_smearing[select_columns])

        n_events_back_after_cut_SEM = predict_proba_SEM[ y_test == 0 ][:,1] > best_cut( y_test, predict_proba_SEM ) 
        n_events_signal_after_cut_SEM = predict_proba_SEM[ y_test == 1 ][:,1] > best_cut(y_test,predict_proba_SEM )
        n_events_SM_after_cut_SEM = predict_proba_SEM[ y_test == 2 ][:,1] > best_cut(y_test,predict_proba_SEM )
        print('Numero de eventos de background depois do corte SEM smearing -->', dataset_sem_smearing[dataset_sem_smearing['label'] == 0]['weight'][ n_events_back_after_cut_SEM ].sum() / test_size )
        print('Numero de eventos de signal depois do corte SEM smearing -->', dataset_sem_smearing[dataset_sem_smearing['label'] == 1]['weight'][ n_events_signal_after_cut_SEM ].sum() / test_size )
        print('Numero de eventos de SM depois do corte SEM smearing -->', dataset_sem_smearing[dataset_sem_smearing['label'] == 2]['weight'][ n_events_SM_after_cut_SEM ].sum() / test_size )
        '''
        n_events_smearing_Signal.append( dset_full[dset_full['label']==1]['weight'][ n_events_signal_after_cut ].sum() / test_size )
        #n_events_Signal.append( dataset_sem_smearing[dataset_sem_smearing['label'] == 1 ]['weight'][n_events_signal_after_cut_SEM].sum() / test_size )

    with h5py.File( '/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/output_smearing' + label_ + '.h5', 'w') as f:
        dset = f.create_dataset( 'dados', data = n_events_smearing_Signal )    

In [ ]:
#smearing_loop( DataSet_Test8_, 500, bst8_binary_logloss_scale_pos_weight, 'anomalo8_parte2' )
#smearing_loop( DataSet_Test7_, 500, bst7_binary_logloss_scale_pos_weight, 'anomalo7_parte2' )
#smearing_loop( DataSet_Test6_, 500, bst6_binary_logloss_scale_pos_weight, 'anomalo6_parte2' )
#smearing_loop( DataSet_Test5_, 500, bst5_binary_logloss_scale_pos_weight, 'anomalo5_parte2' )
#smearing_loop( DataSet_Test4_, 500, bst4_binary_logloss_scale_pos_weight, 'anomalo4_parte2' )
#smearing_loop( DataSet_Test3_, 500, bst3_binary_logloss_scale_pos_weight, 'anomalo3_parte2' )
#smearing_loop( DataSet_Test2_, 500, bst2_binary_logloss_scale_pos_weight, 'anomalo2_parte2' )
#smearing_loop( DataSet_Test1_, 500, bst1_binary_logloss_scale_pos_weight, 'anomalo1_parte2' )

In [ ]:
predict_proba_LGBM8_binary_logloss_scale_pos_weight = bst8_binary_logloss_scale_pos_weight.predict(DataSet_Test8_[select_columns])
print('Predict Proba Anomalo 8 --> \n', predict_proba_LGBM8_binary_logloss_scale_pos_weight)

predict_proba_LGBM7_binary_logloss_scale_pos_weight = bst7_binary_logloss_scale_pos_weight.predict(DataSet_Test7_[select_columns])
print('Predict Proba Anomalo 7 --> \n', predict_proba_LGBM7_binary_logloss_scale_pos_weight)

predict_proba_LGBM6_binary_logloss_scale_pos_weight = bst6_binary_logloss_scale_pos_weight.predict(DataSet_Test6_[select_columns])
print('Predict Proba Anomalo 6 --> \n', predict_proba_LGBM6_binary_logloss_scale_pos_weight)

predict_proba_LGBM5_binary_logloss_scale_pos_weight = bst5_binary_logloss_scale_pos_weight.predict(DataSet_Test5_[select_columns])
print('Predict Proba Anomalo 5 --> \n', predict_proba_LGBM5_binary_logloss_scale_pos_weight)

predict_proba_LGBM1_binary_logloss_scale_pos_weight = bst1_binary_logloss_scale_pos_weight.predict(DataSet_Test1_[select_columns])
print('Predict Proba Anomalo 1 --> \n', predict_proba_LGBM1_binary_logloss_scale_pos_weight)

predict_proba_LGBM2_binary_logloss_scale_pos_weight = bst2_binary_logloss_scale_pos_weight.predict(DataSet_Test2_[select_columns])
print('Predict Proba Anomalo 2 --> \n', predict_proba_LGBM2_binary_logloss_scale_pos_weight)

predict_proba_LGBM3_binary_logloss_scale_pos_weight = bst3_binary_logloss_scale_pos_weight.predict(DataSet_Test3_[select_columns])
print('Predict Proba Anomalo 3 --> \n', predict_proba_LGBM3_binary_logloss_scale_pos_weight)

predict_proba_LGBM4_binary_logloss_scale_pos_weight = bst4_binary_logloss_scale_pos_weight.predict(DataSet_Test4_[select_columns])
print('Predict Proba Anomalo 4 --> \n', predict_proba_LGBM4_binary_logloss_scale_pos_weight)

In [ ]:
y_pred_1 = [np.argmax(line) for line in predict_proba_LGBM1_binary_logloss_scale_pos_weight > best_cut( y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight )]
y_pred_2 = [np.argmax(line) for line in predict_proba_LGBM2_binary_logloss_scale_pos_weight > best_cut( y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight )]
y_pred_3 = [np.argmax(line) for line in predict_proba_LGBM3_binary_logloss_scale_pos_weight > best_cut( y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight )]
y_pred_4 = [np.argmax(line) for line in predict_proba_LGBM4_binary_logloss_scale_pos_weight > best_cut( y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight )]
y_pred_5 = [np.argmax(line) for line in predict_proba_LGBM5_binary_logloss_scale_pos_weight > best_cut( y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight )]
y_pred_6 = [np.argmax(line) for line in predict_proba_LGBM6_binary_logloss_scale_pos_weight > best_cut( y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight )]
y_pred_7 = [np.argmax(line) for line in predict_proba_LGBM7_binary_logloss_scale_pos_weight > best_cut( y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight )]
y_pred_8 = [np.argmax(line) for line in predict_proba_LGBM8_binary_logloss_scale_pos_weight > best_cut( y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight )]


In [ ]:
fig, ax = plt.subplots( 2, 2, figsize=(24,15) )
predict_dados = bst4_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,0].hist( predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test4_weight_backgr/test_size )
ax[0,0].hist( predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=8.0 \times 10^{-6}$', weights = DataSet_Test4_weight_signal/test_size, color = 'green' )
ax[0,0].hist( predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM4_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,0].plot( [ best_cut(y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight),best_cut( y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut4 = predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 0 ][:,1] > best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut4 = predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 1 ][:,1] > best_cut(y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight )
n_events_SM_after_cut4 = predict_proba_LGBM4_binary_logloss_scale_pos_weight[ y_test4 == 2 ][:,1] > best_cut(y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut4 = predict_dados[ predict_dados >= best_cut( y_test4,predict_proba_LGBM4_binary_logloss_scale_pos_weight ) ]

print(" ------------------ Anomalo 4 ------------------ ")
print('Numero de eventos de background depois do corte -->', DataSet_Test4_weight_backgr[ n_events_back_after_cut4 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test4_weight_signal[ n_events_signal_after_cut4].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM4_weight_signal[ n_events_SM_after_cut4 ].sum() / test_size )
print('Recall Score',recall_score(y_test4,y_pred_4, average = 'weighted') )
print('Precision Score',precision_score(y_test4,y_pred_4, average = 'weighted') )

ax[0,0].text(best_cut( y_test4, predict_proba_LGBM4_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,0].set_ylabel( "Events (Log Scale)" )
ax[0,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,0].legend( loc = "upper right", fontsize = 16 )
ax[0,0].set_yscale( 'log' )
ax[0,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,0] )


predict_dados = bst1_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,1].hist( predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test1_weight_backgr/test_size )
ax[0,1].hist( predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=2.0 \times 10^{-5}$', weights = DataSet_Test1_weight_signal/test_size, color = 'green' )
ax[0,1].hist( predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM1_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,1].plot( [ best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight),best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut1 = predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 0 ][:,1] > best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut1 = predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 1 ][:,1] > best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight )
n_events_SM_after_cut1 = predict_proba_LGBM1_binary_logloss_scale_pos_weight[ y_test1 == 2 ][:,1] > best_cut(y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut1 = predict_dados[ predict_dados >= best_cut( y_test1,predict_proba_LGBM1_binary_logloss_scale_pos_weight ) ]

print(' ------------------ Anomalo 1 ------------------ ')
print('Numero de eventos de background depois do corte -->', DataSet_Test1_weight_backgr[ n_events_back_after_cut1 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test1_weight_signal[ n_events_signal_after_cut1].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM1_weight_signal[ n_events_SM_after_cut1 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut1 ) )
print('Recall Score',recall_score(y_test1,y_pred_1, average = 'weighted') )
print('Precision Score',precision_score(y_test1,y_pred_1, average = 'weighted') )

ax[0,1].text(best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,1].legend( loc = "upper right", fontsize = 16 )
ax[0,1].set_yscale( 'log' )
ax[0,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,1] )

predict_dados = bst3_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,0].hist( predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test3_weight_backgr/test_size )
ax[1,0].hist( predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', weights = DataSet_Test3_weight_signal/test_size, color = 'green' )
ax[1,0].hist( predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM3_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,0].plot( [ best_cut(y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight),best_cut(y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut3 = predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 0 ][:,1] >= best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut3 = predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 1 ][:,1] >= best_cut(y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight )
n_events_SM_after_cut3 = predict_proba_LGBM3_binary_logloss_scale_pos_weight[ y_test3 == 2 ][:,1] >= best_cut(y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut3 = predict_dados[ predict_dados >= best_cut( y_test3,predict_proba_LGBM3_binary_logloss_scale_pos_weight ) ]

print(' ------------------ Anomalo 3 ------------------ ')
print('Numero de eventos de background depois do corte -->', DataSet_Test3_weight_backgr[ n_events_back_after_cut3 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test3_weight_signal[ n_events_signal_after_cut3].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM3_weight_signal[ n_events_SM_after_cut3 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut3 ) )
print('Recall Score',recall_score(y_test3,y_pred_3, average = 'weighted') )
print('Precision Score',precision_score(y_test3,y_pred_3, average = 'weighted') )

ax[1,0].text(best_cut( y_test3, predict_proba_LGBM3_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,0].legend( loc = "upper right", fontsize = 16 )
ax[1,0].set_yscale( 'log' )
ax[1,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,0] )

predict_dados = bst2_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,1].hist( predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test2_weight_backgr/test_size )
ax[1,1].hist( predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', weights = DataSet_Test2_weight_signal/test_size, color = 'green' )
ax[1,1].hist( predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standar Model', weights = DataSet_TestSM2_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,1].plot( [ best_cut(y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight),best_cut(y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut2 = predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 0 ][:,1] > best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut2 = predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 1 ][:,1] > best_cut(y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight )
n_events_SM_after_cut2 = predict_proba_LGBM2_binary_logloss_scale_pos_weight[ y_test2 == 2 ][:,1] > best_cut(y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut2 = predict_dados[ predict_dados > best_cut( y_test2,predict_proba_LGBM2_binary_logloss_scale_pos_weight ) ]

print(' ------------------ Anomalo 2 ------------------ ')
print('Numero de eventos de background depois do corte -->', DataSet_Test2_weight_backgr[ n_events_back_after_cut2 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test2_weight_signal[ n_events_signal_after_cut2 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM2_weight_signal[ n_events_SM_after_cut2 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut2 ) )
print('Recall Score',recall_score(y_test2,y_pred_2, average = 'weighted') )
print('Precision Score',precision_score(y_test2,y_pred_2, average = 'weighted') )

ax[1,1].set_xlabel( "Probability" )
ax[1,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,1].legend( loc = "upper right", fontsize = 16 )
ax[1,1].set_yscale( 'log' )
ax[1,1].text(best_cut( y_test2, predict_proba_LGBM2_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,1] )

In [ ]:
fig, ax = plt.subplots( 2, 2, figsize=(24,15) )
predict_dados = bst5_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,0].hist( predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test5_weight_backgr/test_size )
ax[0,0].hist( predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=0.5 \times 10^{-6}$', weights = DataSet_Test5_weight_signal/test_size, color = 'green' )
ax[0,0].hist( predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM5_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,0].plot( [ best_cut(y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight),best_cut( y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut5 = predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 0 ][:,1] > best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut5 = predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 1 ][:,1] > best_cut(y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight )
n_events_SM_after_cut5 = predict_proba_LGBM5_binary_logloss_scale_pos_weight[ y_test5 == 2 ][:,1] > best_cut(y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut5 = predict_dados[ predict_dados >= best_cut( y_test5,predict_proba_LGBM5_binary_logloss_scale_pos_weight ) ]

print("Anomalo 5 ")
print('Numero de eventos de background depois do corte -->', DataSet_Test5_weight_backgr[ n_events_back_after_cut5 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test5_weight_signal[ n_events_signal_after_cut5 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM5_weight_signal[ n_events_SM_after_cut5 ].sum() / test_size )
print('Recall Score',recall_score(y_test5,y_pred_5, average = 'weighted') )
print('Precision Score',precision_score(y_test5,y_pred_5, average = 'weighted') )

ax[0,0].text(best_cut( y_test5, predict_proba_LGBM5_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,0].set_ylabel( "Events (Log Scale)" )
ax[0,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,0].legend( loc = "upper right", fontsize = 16 )
ax[0,0].set_yscale( 'log' )
ax[0,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,0] )


predict_dados = bst6_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[0,1].hist( predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test6_weight_backgr/test_size )
ax[0,1].hist( predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=1.0 \times 10^{-5}$', weights = DataSet_Test6_weight_signal/test_size, color = 'green' )
ax[0,1].hist( predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM6_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,1].plot( [ best_cut(y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight),best_cut(y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight)+0.02],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight)+0.02 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut6 = predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 0 ][:,1] > best_cut( y_test6, predict_proba_LGBM6_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut6 = predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 1 ][:,1] > best_cut(y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight )
n_events_SM_after_cut6 = predict_proba_LGBM6_binary_logloss_scale_pos_weight[ y_test6 == 2 ][:,1] > best_cut(y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut6 = predict_dados[ predict_dados >= best_cut( y_test6,predict_proba_LGBM6_binary_logloss_scale_pos_weight ) ]

print('Anomalo 6')
print('Numero de eventos de background depois do corte -->', DataSet_Test6_weight_backgr[ n_events_back_after_cut6 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test6_weight_signal[ n_events_signal_after_cut6].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM6_weight_signal[ n_events_SM_after_cut6 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut6 ) )
print('Recall Score',recall_score(y_test6,y_pred_6, average = 'weighted') )
print('Precision Score',precision_score(y_test6,y_pred_6, average = 'weighted') )

ax[0,1].text(best_cut( y_test1, predict_proba_LGBM1_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,1].legend( loc = "upper right", fontsize = 16 )
ax[0,1].set_yscale( 'log' )
ax[0,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,1] )

predict_dados = bst7_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,0].hist( predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test7_weight_backgr/test_size )
ax[1,0].hist( predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', weights = DataSet_Test7_weight_signal/test_size, color = 'green' )
ax[1,0].hist( predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM7_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,0].plot( [ best_cut(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight),best_cut(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut7 = predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 0 ][:,1] >= best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut7 = predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 1 ][:,1] >= best_cut(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight )
n_events_SM_after_cut7 = predict_proba_LGBM7_binary_logloss_scale_pos_weight[ y_test7 == 2 ][:,1] >= best_cut(y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut7 = predict_dados[ predict_dados >= best_cut( y_test7,predict_proba_LGBM7_binary_logloss_scale_pos_weight ) ]

print('Anomalo 7')
print('Numero de eventos de background depois do corte -->', DataSet_Test7_weight_backgr[ n_events_back_after_cut7 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test7_weight_signal[ n_events_signal_after_cut7].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM7_weight_signal[ n_events_SM_after_cut7 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut7 ) )
print('Recall Score',recall_score(y_test7,y_pred_7, average = 'weighted') )
print('Precision Score',precision_score(y_test7,y_pred_7, average = 'weighted') )

ax[1,0].text(best_cut( y_test7, predict_proba_LGBM7_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,0].legend( loc = "upper right", fontsize = 16 )
ax[1,0].set_yscale( 'log' )
ax[1,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,0] )

predict_dados = bst8_binary_logloss_scale_pos_weight.predict(data_set_dados_multirp[ select_columns ])[:,1]
bins = 25

hbgt_lgb = ax[1,1].hist( predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 0 ][:,1], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test8_weight_backgr/test_size )
ax[1,1].hist( predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 1 ][:,1], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', weights = DataSet_Test8_weight_signal/test_size, color = 'green' )
ax[1,1].hist( predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 2 ][:,1], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standar Model', weights = DataSet_TestSM8_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,1].plot( [ best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight),best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight)],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight) ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut( y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut8 = predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 0 ][:,1] > best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight ) 
n_events_signal_after_cut8 = predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 1 ][:,1] > best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight )
n_events_SM_after_cut8 = predict_proba_LGBM8_binary_logloss_scale_pos_weight[ y_test8 == 2 ][:,1] > best_cut(y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight )
n_eventos_Data_after_cut8 = predict_dados[ predict_dados > best_cut( y_test8,predict_proba_LGBM8_binary_logloss_scale_pos_weight ) ]

print('Anomalo 8')
print('Numero de eventos de background depois do corte -->', DataSet_Test8_weight_backgr[ n_events_back_after_cut8 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test8_weight_signal[ n_events_signal_after_cut8 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM8_weight_signal[ n_events_SM_after_cut8 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut8 ) )
print('Recall Score',recall_score(y_test8,y_pred_8, average = 'weighted') )
print('Precision Score',precision_score(y_test8,y_pred_8, average = 'weighted') ) 

ax[1,1].set_xlabel( "Probability" )
ax[1,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,1].legend( loc = "upper right", fontsize = 16 )
ax[1,1].set_yscale( 'log' )
ax[1,1].text(best_cut( y_test8, predict_proba_LGBM8_binary_logloss_scale_pos_weight )+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[1,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,1] )